In [21]:
import requests
import json
from pprint import pprint
import pandas as pd
import spotipy as sp
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler


#import spotify keys
from config import client_id
from config import client_secret

# Creating Access Token for Spotify API

In [2]:
#generate access token for spotify api
#from: https://stmorse.github.io/journal/spotify-api.html

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
#from: https://stmorse.github.io/journal/spotify-api.html

#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# Requesting Input from User

In [4]:
# need to add instructions or code to remove punctuations for search to work
search_track = input("Please name a song: ")

Please name a song:  brown eyed girl


In [5]:
search_artist = input("Please name that song's artist: ")

Please name that song's artist:  van morrison


In [ ]:
# region_preference = input("Please select a region: ")

# Searching for Matching Track Using Spotify API

In [6]:
response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()

In [7]:
pprint(response)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3Abrown+eyed+girl+artist%3Avan+morrison&type=artist&offset=0&limit=1',
             'items': [],
             'limit': 1,
             'next': None,
             'offset': 0,
             'previous': None,
             'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Abrown+eyed+girl+artist%3Avan+morrison&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/44NX2ffIYHr6D4n7RaZF7A'},
                                              'href': 'https://api.spotify.com/v1/artists/44NX2ffIYHr6D4n7RaZF7A',
                                              'id': '44NX2ffIYHr6D4n7RaZF7A',
                                              'name': 'Van Morrison',
                                              'type': 'artist',
                                              

In [8]:
pprint(response['tracks']['items'][0]["id"])

'3yrSvpt2l1xhsV9Em88Pul'


In [9]:
track_id = response['tracks']['items'][0]["id"]

# Pulling Track Features from Spotify API

In [10]:
#base URL for Spotify API
base_url = 'https://api.spotify.com/v1/'

track_response = requests.get(base_url + 'audio-features/' + track_id, headers=headers).json()

In [11]:
#empty lists to store song data
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
id_num = []
duration_mins = []
time_signature = []

In [12]:
#fill in track info for each audio feature / empty list
danceability.append(track_response["danceability"])
energy.append(track_response["energy"])
key.append(track_response["key"])
loudness.append(track_response["loudness"])
mode.append(track_response["mode"])
speechiness.append(track_response["speechiness"])
acousticness.append(track_response["acousticness"])
instrumentalness.append(track_response["instrumentalness"])
liveness.append(track_response["liveness"])
valence.append(track_response["valence"])
tempo.append(track_response["tempo"])
id_num.append(track_response["id"])
#duration recorded in api in milliseconds, converting to minutes here
duration_mins.append(round((track_response["duration_ms"] / 60000),2))
time_signature.append(track_response["time_signature"])    

In [13]:
#create a dictionary to hold data gathered from the api
track_dict = {
    "track_id": id_num,
    "danceability": danceability,
    "energy": energy,
    "key": key,
    "loudness": loudness,
    "mode": mode,
    "speechiness": speechiness,
    "acousticness": acousticness,
    "instrumentalness": instrumentalness,
    "liveness": liveness,
    "valence": valence,
    "tempo": tempo,
    "duration_mins": duration_mins,
    "time_signature": time_signature,
    
}
#convert dictionary to a dataframe
input_track_df = pd.DataFrame(track_dict)
input_track_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3yrSvpt2l1xhsV9Em88Pul,0.491,0.583,7,-10.964,1,0.0376,0.185,0,0.406,0.908,150.566,3.06,4


# Importing Global Top 200 Chart for Testing

In [25]:
#import global top songs csv and save as a df
global2022_df = pd.read_csv("resources/regional-global-weekly-2022-10-06.csv")
global2022_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),EMI,1,1,2,53478156
1,2,spotify:track:4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Parlophone UK,2,2,6,33516367
2,3,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,3,27,32910077
3,4,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",DALE PLAY Records,1,4,13,31691825
4,5,spotify:track:5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Sony Music Latin/La Industria,5,5,14,29821181


In [26]:
#extracting track id (removing "spotify:track:" from track id in csv)
track_id_series = global2022_df.uri.str[14:]

#convert to df and rename column
track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})

#merge dfs
global2022_df = global2022_df.merge(track_id_df, how="inner", left_index=True, right_index=True)
global2022_df = global2022_df[["track_id", "artist_names", "track_name"]]
global2022_df = global2022_df.astype({'track_id':'string','artist_names':'string','track_name':'string'})

In [27]:
global2022_df.head()

,track_id,artist_names,track_name
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras)
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue)
2,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52"
4,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata


In [28]:
#empty lists to store song data
global_danceability = []
global_energy = []
global_key = []
global_loudness = []
global_mode = []
global_speechiness = []
global_acousticness = []
global_instrumentalness = []
global_liveness = []
global_valence = []
global_tempo = []
global_id_num = []
global_duration_mins = []
global_time_signature = []

#loop through dataframe, use track id to look up songs in spotify api and grab each one's audio features
for t in global2022_df["track_id"]:
    global_response = requests.get(base_url + 'audio-features/' + t, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    global_danceability.append(global_response["danceability"])
    global_energy.append(global_response["energy"])
    global_key.append(global_response["key"])
    global_loudness.append(global_response["loudness"])
    global_mode.append(global_response["mode"])
    global_speechiness.append(global_response["speechiness"])
    global_acousticness.append(global_response["acousticness"])
    global_instrumentalness.append(global_response["instrumentalness"])
    global_liveness.append(global_response["liveness"])
    global_valence.append(global_response["valence"])
    global_tempo.append(global_response["tempo"])
    global_id_num.append(global_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    global_duration_mins.append(round((global_response["duration_ms"] / 60000),2))
    global_time_signature.append(global_response["time_signature"])   
    
#create a dictionary to hold data gathered from the api
global_songs_dict = {
    "track_id": global_id_num,
    "danceability": global_danceability,
    "energy": global_energy,
    "key": global_key,
    "loudness": global_loudness,
    "mode": global_mode,
    "speechiness": global_speechiness,
    "acousticness": global_acousticness,
    "instrumentalness": global_instrumentalness,
    "liveness": global_liveness,
    "valence": global_valence,
    "tempo": global_tempo,
    "duration_mins": global_duration_mins,
    "time_signature": global_time_signature,
    
}
#convert dictionary to a dataframe
gloabl_info_df = pd.DataFrame(global_songs_dict)
gloabl_info_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.266,0.238,131.121,2.62,4
1,4uUG5RXrOk84mYEfFvj3cK,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.371,0.304,128.040,2.92,4
2,4Dvkj6JhhA12EX05fT7y2e,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.311,0.662,173.930,2.79,4
3,2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.230,0.550,128.033,3.32,4
4,5ww2BF9slyYgNOk37BlC4u,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.218,0.850,124.980,2.71,4


In [29]:
global_df = global2022_df.merge(gloabl_info_df, how="inner")
global_df

,track_id,artist_names,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
2,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,2.79,4
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,3.32,4
4,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,2.71,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4woTEX1wYOTGDqNXuavlRC,"Eminem, Dina Rae",Superman,0.802,0.755,4,-3.138,0,0.0564,0.02150,0.000000,0.2950,0.580,130.027,5.84,4
196,003vvx7Niy0yvhvHt4a68B,The Killers,Mr. Brightside,0.352,0.911,1,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,3.72,4
197,2K7xn816oNHJZ0aVqdQsha,The Neighbourhood,Softcore,0.575,0.568,9,-5.509,0,0.0300,0.04840,0.000417,0.2860,0.370,93.986,3.44,4
198,4gzsuuZypVbxs0Af1LSZyB,"Tainy, Bad Bunny, Julieta Venegas",Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),0.639,0.703,4,-6.330,0,0.0794,0.08890,0.000002,0.0865,0.138,169.888,3.46,4


# Combining Input Track Features With Chart Tracks and Scaling Data

In [78]:
#creating df with input track features as row 0, and rest of chart tracks below
chart_with_input_track = pd.concat([input_track_df,gloabl_info_df.loc[:]]).reset_index(drop=True)
chart_with_input_track.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3yrSvpt2l1xhsV9Em88Pul,0.491,0.583,7,-10.964,1,0.0376,0.18500,0.000000,0.406,0.908,150.566,3.06,4
1,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.266,0.238,131.121,2.62,4
2,4uUG5RXrOk84mYEfFvj3cK,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.371,0.304,128.040,2.92,4
3,4Dvkj6JhhA12EX05fT7y2e,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.311,0.662,173.930,2.79,4
4,2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.230,0.550,128.033,3.32,4


In [79]:
scaler = StandardScaler()
# scaler = MinMaxScaler()

chart_scaled = scaler.fit_transform(chart_with_input_track.iloc[:, 1:14])
chart_scaled_df = pd.DataFrame(chart_scaled, columns=chart_with_input_track.iloc[:, 1:14].columns)
chart_scaled_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,-1.103056,-0.468579,0.559717,-2.019389,0.838683,-0.577898,-0.188446,-0.210507,1.899082,1.734549,0.912357,-0.543356,0.250209
1,0.361293,-1.179718,-0.786013,-0.533387,0.838683,0.002860,-0.906795,-0.210450,0.771291,-1.143919,0.258939,-1.140082,0.250209
2,-0.643395,1.978765,0.559717,0.999402,-1.192345,-0.617171,-0.945093,-0.210418,1.617134,-0.860368,0.155407,-0.733223,0.250209
3,-0.912625,0.479607,0.290571,0.310020,-1.192345,-0.362494,0.467257,-0.197787,1.133795,0.677679,1.697466,-0.909529,0.250209
4,-0.249400,0.806347,-0.786013,0.223071,0.838683,-0.501733,-0.908883,0.205118,0.481287,0.196502,0.155171,-0.190745,0.250209


# Creating CSVs of Regional Top 200 Charts

In [30]:
#import regional top songs csvs and save as individual dfs
regional_files = {"usa":"regional-usa-weekly-2022-10-06.csv", "vietnam":"regional-vietnam-weekly-2022-10-06.csv","venezuela":"regional-venezuela-weekly-2022-10-06.csv",
"uruguay":"regional-uruguay-weekly-2022-10-06.csv","united_kingdom":"regional-united_kingdom-weekly-2022-10-06.csv","ukraine":"regional-ukraine-weekly-2022-10-06.csv",
"uae":"regional-uae-weekly-2022-10-06.csv","turkey":"regional-turkey-weekly-2022-10-06.csv","thailand":"regional-thailand-weekly-2022-10-06.csv",
"taiwan":"regional-taiwan-weekly-2022-10-06.csv","switzerland":"regional-switzerland-weekly-2022-10-06.csv","sweden":"regional-sweden-weekly-2022-10-06.csv",
"spain":"regional-spain-weekly-2022-10-06.csv","south_korea":"regional-south_korea-weekly-2022-10-06.csv","south_africa":"regional-south_africa-weekly-2022-10-06.csv",
"slovakia":"regional-slovakia-weekly-2022-10-06.csv","singapore":"regional-singapore-weekly-2022-10-06.csv","saudi_arabia":"regional-saudi_arabia-weekly-2022-10-06.csv",
"romania":"regional-romania-weekly-2022-10-06.csv","portugal":"regional-portugal-weekly-2022-10-06.csv","poland":"regional-poland-weekly-2022-10-06.csv",
"philippines":"regional-philippines-weekly-2022-10-06.csv","peru":"regional-peru-weekly-2022-10-06.csv","paraguay":"regional-paraguay-weekly-2022-10-06.csv",
"panama":"regional-panama-weekly-2022-10-06.csv","pakistan":"regional-pakistan-weekly-2022-10-06.csv","norway":"regional-norway-weekly-2022-10-06.csv",
"nigeria":"regional-nigeria-weekly-2022-10-06.csv","nicaragua":"regional-nicaragua-weekly-2022-10-06.csv","new_zealand":"regional-new_zealand-weekly-2022-10-06.csv",
"netherlands":"regional-netherlands-weekly-2022-10-06.csv","morocco":"regional-morocco-weekly-2022-10-06.csv","mexico":"regional-mexico-weekly-2022-10-06.csv",
"malaysia":"regional-malaysia-weekly-2022-10-06.csv","luxembourg":"regional-luxembourg-weekly-2022-10-06.csv","lithuania":"regional-lithuania-weekly-2022-10-06.csv",
"latvia":"regional-latvia-weekly-2022-10-06.csv","kazakhstan":"regional-kazakhstan-weekly-2022-10-06.csv","japan":"regional-japan-weekly-2022-10-06.csv",
"italy":"regional-italy-weekly-2022-10-06.csv","israel":"regional-israel-weekly-2022-10-06.csv","ireland":"regional-ireland-weekly-2022-10-06.csv",
"indonesia":"regional-indonesia-weekly-2022-10-06.csv","india":"regional-india-weekly-2022-10-06.csv","iceland":"regional-iceland-weekly-2022-10-06.csv",
"hungary":"regional-hungary-weekly-2022-10-06.csv","hong_kong":"regional-hong_kong-weekly-2022-10-06.csv","honduras":"regional-honduras-weekly-2022-10-06.csv",
"guatemala":"regional-guatemala-weekly-2022-10-06.csv","greece":"regional-greece-weekly-2022-10-06.csv","germany":"regional-germany-weekly-2022-10-06.csv",
"france":"regional-france-weekly-2022-10-06.csv","finland":"regional-finland-weekly-2022-10-06.csv","estonia":"regional-estonia-weekly-2022-10-06.csv",
"el_salvador":"regional-el_salvador-weekly-2022-10-06.csv","egypt":"regional-egypt-weekly-2022-10-06.csv","ecuador":"regional-ecuador-weekly-2022-10-06.csv",
"dominican_republic":"regional-dominican_republic-weekly-2022-10-06.csv","denmark":"regional-denmark-weekly-2022-10-06.csv","czech_republic":"regional-czech_republic-weekly-2022-10-06.csv",
"cyprus":"regional-cyprus-weekly-2022-10-06.csv","costa_rica":"regional-costa_rica-weekly-2022-10-06.csv","colombia":"regional-colombia-weekly-2022-10-06.csv",
"chile":"regional-chile-weekly-2022-10-06.csv","canada":"regional-canada-weekly-2022-10-06.csv","bulgaria":"regional-bulgaria-weekly-2022-10-06.csv",
"brazil":"regional-brazil-weekly-2022-10-06.csv","bolivia":"regional-bolivia-weekly-2022-10-06.csv","belgium":"regional-belgium-weekly-2022-10-06.csv",
"belarus":"regional-belarus-weekly-2022-10-06.csv","austria":"regional-austria-weekly-2022-10-06.csv","australia":"regional-australia-weekly-2022-10-06.csv",
"argentina":"regional-argentina-weekly-2022-10-06.csv"}

d = {}

for key in regional_files:
    # print(key, regional_files[key])
    d[key] = pd.read_csv(f'resources/{regional_files[key]}') 
    
    #extracting track id (removing "spotify:track:" from track id in csv)
    track_id_series = d[key].uri.str[14:]
    
    #convert to df and rename column
    track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})
    
    #merge dfs
    d[key] = d[key].merge(track_id_df, how="inner", left_index=True, right_index=True)
    d[key] = d[key][["track_id", "artist_names", "track_name"]]
    #add column for region
    d[key]['region'] = key.capitalize()
    #convert values to string
    d[key] =d[key].astype({'track_id':'string','artist_names':'string','track_name':'string','region':'string'})

In [31]:
d['argentina']

,track_id,artist_names,track_name,region
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Argentina
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Argentina
2,45dlaEXrXW47TVbrQ4gK7E,"Rei, Callejero Fino",Tu Turrito,Argentina
3,3Ne5uYDFzHEFYAPbaBMIck,Maria Becerra,AUTOMÁTICO,Argentina
4,53tfEupEzQRtVFOeZvk7xq,ROSALÍA,DESPECHÁ,Argentina
...,...,...,...,...
195,6nAlJuLlRlpAcq30xIqKJs,Plan B,¿Por Que Te Demoras?,Argentina
196,4iaNmAvcZ6sPGvgZ6vQQdx,"Quevedo, Linton",Ahora y Siempre,Argentina
197,65DBZofI0b79kfHTcWWDuU,Soda Stereo,Trátame Suavemente - Remasterizado 2007,Argentina
198,0WuKq2LiraBFney78dzwoc,Ciro y los Persas,Mírenla,Argentina


In [32]:
regional_list = d.keys()
regional_list

dict_keys(['usa', 'vietnam', 'venezuela', 'uruguay', 'united_kingdom', 'ukraine', 'uae', 'turkey', 'thailand', 'taiwan', 'switzerland', 'sweden', 'spain', 'south_korea', 'south_africa', 'slovakia', 'singapore', 'saudi_arabia', 'romania', 'portugal', 'poland', 'philippines', 'peru', 'paraguay', 'panama', 'pakistan', 'norway', 'nigeria', 'nicaragua', 'new_zealand', 'netherlands', 'morocco', 'mexico', 'malaysia', 'luxembourg', 'lithuania', 'latvia', 'kazakhstan', 'japan', 'italy', 'israel', 'ireland', 'indonesia', 'india', 'iceland', 'hungary', 'hong_kong', 'honduras', 'guatemala', 'greece', 'germany', 'france', 'finland', 'estonia', 'el_salvador', 'egypt', 'ecuador', 'dominican_republic', 'denmark', 'czech_republic', 'cyprus', 'costa_rica', 'colombia', 'chile', 'canada', 'bulgaria', 'brazil', 'bolivia', 'belgium', 'belarus', 'austria', 'australia', 'argentina'])

In [33]:
len(regional_list)

73

# Argentina Top Chart Track Features

In [34]:
#empty lists to store song data
argentina_danceability = []
argentina_energy = []
argentina_key = []
argentina_loudness = []
argentina_mode = []
argentina_speechiness = []
argentina_acousticness = []
argentina_instrumentalness = []
argentina_liveness = []
argentina_valence = []
argentina_tempo = []
argentina_id_num = []
argentina_duration_mins = []
argentina_time_signature = []

for i in d['argentina']["track_id"]:
    argentina_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    argentina_danceability.append(argentina_response["danceability"])
    argentina_energy.append(argentina_response["energy"])
    argentina_key.append(argentina_response["key"])
    argentina_loudness.append(argentina_response["loudness"])
    argentina_mode.append(argentina_response["mode"])
    argentina_speechiness.append(argentina_response["speechiness"])
    argentina_acousticness.append(argentina_response["acousticness"])
    argentina_instrumentalness.append(argentina_response["instrumentalness"])
    argentina_liveness.append(argentina_response["liveness"])
    argentina_valence.append(argentina_response["valence"])
    argentina_tempo.append(argentina_response["tempo"])
    argentina_id_num.append(argentina_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    argentina_duration_mins.append(round((argentina_response["duration_ms"] / 60000),2))
    argentina_time_signature.append(argentina_response["time_signature"])  

    # time.sleep(1)

argentina_dict = {}
#create a dictionary to hold data gathered from the api
argentina_dict = {
    "track_id": argentina_id_num,
    "danceability": argentina_danceability,
    "energy": argentina_energy,
    "key": argentina_key,
    "loudness": argentina_loudness,
    "mode": argentina_mode,
    "speechiness": argentina_speechiness,
    "acousticness": argentina_acousticness,
    "instrumentalness": argentina_instrumentalness,
    "liveness": argentina_liveness,
    "valence": argentina_valence,
    "tempo": argentina_tempo,
    "duration_mins": argentina_duration_mins,
    "time_signature": argentina_time_signature
}

#convert dictionary to a dataframe
argentina_info_df = pd.DataFrame(argentina_dict)
argentina_info_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,45dlaEXrXW47TVbrQ4gK7E,0.838,0.545,9,-6.100,0,0.0604,0.0966,0.000000,0.0927,0.428,90.023,2.75,4
3,3Ne5uYDFzHEFYAPbaBMIck,0.680,0.808,10,-3.854,0,0.2240,0.0911,0.000002,0.2050,0.868,176.131,2.92,4
4,53tfEupEzQRtVFOeZvk7xq,0.914,0.622,7,-6.538,1,0.0892,0.1460,0.000017,0.0632,0.770,130.040,2.61,4


In [35]:
argentina_tracks = d['argentina']
argentina_df = argentina_tracks.merge(argentina_info_df, how="inner")
argentina_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Argentina,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Argentina,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,45dlaEXrXW47TVbrQ4gK7E,"Rei, Callejero Fino",Tu Turrito,Argentina,0.838,0.545,9,-6.100,0,0.0604,0.0966,0.000000,0.0927,0.428,90.023,2.75,4
3,3Ne5uYDFzHEFYAPbaBMIck,Maria Becerra,AUTOMÁTICO,Argentina,0.680,0.808,10,-3.854,0,0.2240,0.0911,0.000002,0.2050,0.868,176.131,2.92,4
4,53tfEupEzQRtVFOeZvk7xq,ROSALÍA,DESPECHÁ,Argentina,0.914,0.622,7,-6.538,1,0.0892,0.1460,0.000017,0.0632,0.770,130.040,2.61,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6nAlJuLlRlpAcq30xIqKJs,Plan B,¿Por Que Te Demoras?,Argentina,0.891,0.780,2,-6.416,0,0.0701,0.0398,0.000000,0.0548,0.931,95.998,3.18,4
196,4iaNmAvcZ6sPGvgZ6vQQdx,"Quevedo, Linton",Ahora y Siempre,Argentina,0.659,0.629,4,-7.860,1,0.0825,0.7160,0.000000,0.4220,0.416,118.964,2.49,4
197,65DBZofI0b79kfHTcWWDuU,Soda Stereo,Trátame Suavemente - Remasterizado 2007,Argentina,0.738,0.800,11,-3.366,0,0.0283,0.0430,0.054300,0.5950,0.532,122.858,3.35,4
198,0WuKq2LiraBFney78dzwoc,Ciro y los Persas,Mírenla,Argentina,0.632,0.717,2,-6.924,1,0.0272,0.0253,0.000661,0.1910,0.832,161.011,4.27,4


# Australia Top Chart Track Features

In [36]:
#empty lists to store song data
australia_danceability = []
australia_energy = []
australia_key = []
australia_loudness = []
australia_mode = []
australia_speechiness = []
australia_acousticness = []
australia_instrumentalness = []
australia_liveness = []
australia_valence = []
australia_tempo = []
australia_id_num = []
australia_duration_mins = []
australia_time_signature = []

for i in d['australia']["track_id"]:
    australia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    australia_danceability.append(australia_response["danceability"])
    australia_energy.append(australia_response["energy"])
    australia_key.append(australia_response["key"])
    australia_loudness.append(australia_response["loudness"])
    australia_mode.append(australia_response["mode"])
    australia_speechiness.append(australia_response["speechiness"])
    australia_acousticness.append(australia_response["acousticness"])
    australia_instrumentalness.append(australia_response["instrumentalness"])
    australia_liveness.append(australia_response["liveness"])
    australia_valence.append(australia_response["valence"])
    australia_tempo.append(australia_response["tempo"])
    australia_id_num.append(australia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    australia_duration_mins.append(round((australia_response["duration_ms"] / 60000),2))
    australia_time_signature.append(australia_response["time_signature"])  

    # time.sleep(1)

australia_dict = {}
#create a dictionary to hold data gathered from the api
australia_dict = {
    "track_id": australia_id_num,
    "danceability": australia_danceability,
    "energy": australia_energy,
    "key": australia_key,
    "loudness": australia_loudness,
    "mode": australia_mode,
    "speechiness": australia_speechiness,
    "acousticness": australia_acousticness,
    "instrumentalness": australia_instrumentalness,
    "liveness": australia_liveness,
    "valence": australia_valence,
    "tempo": australia_tempo,
    "duration_mins": australia_duration_mins,
    "time_signature": australia_time_signature
}

#convert dictionary to a dataframe
australia_info_df = pd.DataFrame(australia_dict)
australia_info_df.head()


australia_tracks = d['australia']
australia_df = australia_tracks.merge(australia_info_df, how="inner")
australia_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Australia,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Australia,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
2,4h9wh7iOZ0GGn8QVp4RAOB,OneRepublic,I Ain't Worried,Australia,0.704,0.797,0,-5.927,1,0.0475,0.08260,0.000745,0.0546,0.825,139.994,2.47,4
3,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,Australia,0.686,0.494,1,-7.093,1,0.0355,0.61300,0.000058,0.4020,0.700,168.946,3.87,4
4,2yjlYDiNiQkdxVqTlaSrlX,Nicki Minaj,Super Freaky Girl,Australia,0.951,0.878,2,-2.565,1,0.2110,0.05090,0.000016,0.3210,0.923,133.014,2.85,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0d28khcov6AiegSCpG5TuT,Gorillaz,Feel Good Inc.,Australia,0.818,0.705,6,-6.679,1,0.1770,0.00836,0.002330,0.6130,0.772,138.559,3.71,4
196,0y60itmpH0aPKsFiGxmtnh,WILLOW,Wait a Minute!,Australia,0.764,0.705,3,-5.279,0,0.0278,0.03710,0.000019,0.0943,0.672,101.003,3.28,4
197,49x8Zp5wDjoquBE1THLLCV,The Temper Trap,Sweet Disposition,Australia,0.545,0.810,2,-6.899,1,0.0443,0.07010,0.197000,0.1220,0.305,129.138,3.86,4
198,5TXDeTFVRVY7Cvt0Dw4vWW,XXXTENTACION,Revenge,Australia,0.746,0.251,11,-16.169,0,0.2590,0.78200,0.002030,0.1060,0.180,139.999,2.00,4


# Austria Top Char# Austria Top Chart Track Featurest Track Features

In [37]:
#empty lists to store song data
austria_danceability = []
austria_energy = []
austria_key = []
austria_loudness = []
austria_mode = []
austria_speechiness = []
austria_acousticness = []
austria_instrumentalness = []
austria_liveness = []
austria_valence = []
austria_tempo = []
austria_id_num = []
austria_duration_mins = []
austria_time_signature = []

for i in d['austria']["track_id"]:
    austria_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    austria_danceability.append(austria_response["danceability"])
    austria_energy.append(austria_response["energy"])
    austria_key.append(austria_response["key"])
    austria_loudness.append(austria_response["loudness"])
    austria_mode.append(austria_response["mode"])
    austria_speechiness.append(austria_response["speechiness"])
    austria_acousticness.append(austria_response["acousticness"])
    austria_instrumentalness.append(austria_response["instrumentalness"])
    austria_liveness.append(austria_response["liveness"])
    austria_valence.append(austria_response["valence"])
    austria_tempo.append(austria_response["tempo"])
    austria_id_num.append(austria_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    austria_duration_mins.append(round((austria_response["duration_ms"] / 60000),2))
    austria_time_signature.append(austria_response["time_signature"])  

    # time.sleep(1)

austria_dict = {}
#create a dictionary to hold data gathered from the api
austria_dict = {
    "track_id": austria_id_num,
    "danceability": austria_danceability,
    "energy": austria_energy,
    "key": austria_key,
    "loudness": austria_loudness,
    "mode": austria_mode,
    "speechiness": austria_speechiness,
    "acousticness": austria_acousticness,
    "instrumentalness": austria_instrumentalness,
    "liveness": austria_liveness,
    "valence": austria_valence,
    "tempo": austria_tempo,
    "duration_mins": austria_duration_mins,
    "time_signature": austria_time_signature
}

#convert dictionary to a dataframe
austria_info_df = pd.DataFrame(austria_dict)
austria_info_df.head()


austria_tracks = d['austria']
austria_df = austria_tracks.merge(austria_info_df, how="inner")
austria_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,7sJNeWTXkZCEC7OnFMpGWL,"Luciano, Aitch, BIA",Bamba (feat. Aitch & BIA),Austria,0.797,0.791,10,-6.281,1,0.3300,0.11900,0.000000,0.1130,0.828,138.026,3.39,4
1,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Austria,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
2,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Austria,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
3,1Fb8gCHRkmNAaN8vytd5vg,"Luciano, Central Cee",West Connect (feat. Central Cee),Austria,0.920,0.571,10,-7.133,0,0.4940,0.06810,0.000000,0.0969,0.465,139.990,3.54,4
4,6yeZM8gyykjaf8w0GqRfK4,Nina Chuba,Wildberry Lillet,Austria,0.890,0.521,6,-7.048,0,0.1690,0.07800,0.000000,0.0958,0.646,139.981,2.20,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,7iL6o9tox1zgHpKUfh9vuC,50 Cent,In Da Club,Austria,0.899,0.713,6,-2.752,0,0.3660,0.25500,0.000000,0.0708,0.777,90.051,3.22,4
196,3hRV0jL3vUpRrcy398teAU,Lord Huron,The Night We Met,Austria,0.544,0.369,2,-9.514,1,0.0380,0.96900,0.279000,0.6390,0.102,87.010,3.47,3
197,0e8nrvls4Qqv5Rfa2UhqmO,Lil Nas X,THATS WHAT I WANT,Austria,0.737,0.846,1,-4.510,0,0.2200,0.00614,0.000000,0.0486,0.546,87.981,2.40,4
198,5STeQZH1Ahz3h0X9Njaij7,t-low,Crashen,Austria,0.649,0.733,11,-8.462,0,0.3000,0.06790,0.000000,0.1220,0.498,150.874,2.97,4


# Belarus Top Chart Track Features

In [38]:
#empty lists to store song data
belarus_danceability = []
belarus_energy = []
belarus_key = []
belarus_loudness = []
belarus_mode = []
belarus_speechiness = []
belarus_acousticness = []
belarus_instrumentalness = []
belarus_liveness = []
belarus_valence = []
belarus_tempo = []
belarus_id_num = []
belarus_duration_mins = []
belarus_time_signature = []

for i in d['belarus']["track_id"]:
    belarus_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    belarus_danceability.append(belarus_response["danceability"])
    belarus_energy.append(belarus_response["energy"])
    belarus_key.append(belarus_response["key"])
    belarus_loudness.append(belarus_response["loudness"])
    belarus_mode.append(belarus_response["mode"])
    belarus_speechiness.append(belarus_response["speechiness"])
    belarus_acousticness.append(belarus_response["acousticness"])
    belarus_instrumentalness.append(belarus_response["instrumentalness"])
    belarus_liveness.append(belarus_response["liveness"])
    belarus_valence.append(belarus_response["valence"])
    belarus_tempo.append(belarus_response["tempo"])
    belarus_id_num.append(belarus_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    belarus_duration_mins.append(round((belarus_response["duration_ms"] / 60000),2))
    belarus_time_signature.append(belarus_response["time_signature"])  

    # time.sleep(1)

belarus_dict = {}
#create a dictionary to hold data gathered from the api
belarus_dict = {
    "track_id": belarus_id_num,
    "danceability": belarus_danceability,
    "energy": belarus_energy,
    "key": belarus_key,
    "loudness": belarus_loudness,
    "mode": belarus_mode,
    "speechiness": belarus_speechiness,
    "acousticness": belarus_acousticness,
    "instrumentalness": belarus_instrumentalness,
    "liveness": belarus_liveness,
    "valence": belarus_valence,
    "tempo": belarus_tempo,
    "duration_mins": belarus_duration_mins,
    "time_signature": belarus_time_signature
}

#convert dictionary to a dataframe
belarus_info_df = pd.DataFrame(belarus_dict)
belarus_info_df.head()


belarus_tracks = d['belarus']
belarus_df = belarus_info_df.merge(belarus_tracks, how="inner")
belarus_df

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,artist_names,track_name,region
0,2uNrriUaePhbbmsXT8HQJU,0.808,0.688,7,-8.964,0,0.0493,0.115000,0.025100,0.1140,0.377,120.039,2.30,4,INSTASAMKA,POPSTAR,Belarus
1,4oHEJ5KPf9hW1pDyQYIRIw,0.931,0.696,7,-6.656,1,0.1870,0.336000,0.000025,0.3390,0.357,134.005,2.21,4,"Aarne, BUSHIDO ZHO, ANIKV",Тесно,Belarus
2,3pzr7iKQpV9QjLGcLcd2ic,0.764,0.700,2,-4.462,0,0.1170,0.113000,0.000000,0.1330,0.273,145.011,2.04,4,"uglystephan, Scally Milano",Заберу,Belarus
3,6BfnNid8w0UyZqzR2mx1HB,0.843,0.774,10,-3.998,0,0.0612,0.008130,0.000000,0.0668,0.391,101.982,3.03,4,"Молодой Платон, Poshlaya Molly, Yanix","DON'T PLAY, BAE",Belarus
4,21LIoC4ZxT3F8xq9gidNbX,0.643,0.573,1,-6.992,1,0.0466,0.012600,0.000000,0.2010,0.700,139.940,1.80,4,xxxmanera,Never Broke Again,Belarus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0IjCnXqJTlPElz8HmIiO9p,0.979,0.553,7,-5.955,1,0.0567,0.013400,0.000000,0.1130,0.648,116.071,2.40,4,"The Limba, MORGENSHTERN",Известным,Belarus
196,52TAW7MgoBhjx0ESJBvhs3,0.794,0.606,1,-5.963,1,0.0892,0.026900,0.000000,0.3200,0.516,137.934,2.75,4,"LOVV66, blago white",Говорят чо,Belarus
197,6KM7gePYUE2Xefjp4g9L2s,0.700,0.749,0,-6.288,0,0.1400,0.017900,0.000000,0.1550,0.515,175.111,1.55,4,Кишлак,Самый лучший день,Belarus
198,1zdbpQXkC4oHNiYiuLrbQZ,0.621,0.866,11,-4.978,0,0.0375,0.000625,0.000001,0.0849,0.333,124.093,2.59,4,дипинс,Этажи,Belarus


# Belgium Top Chart Track Features

In [39]:
#empty lists to store song data
belgium_danceability = []
belgium_energy = []
belgium_key = []
belgium_loudness = []
belgium_mode = []
belgium_speechiness = []
belgium_acousticness = []
belgium_instrumentalness = []
belgium_liveness = []
belgium_valence = []
belgium_tempo = []
belgium_id_num = []
belgium_duration_mins = []
belgium_time_signature = []

for i in d['belgium']["track_id"]:
    belgium_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    belgium_danceability.append(belgium_response["danceability"])
    belgium_energy.append(belgium_response["energy"])
    belgium_key.append(belgium_response["key"])
    belgium_loudness.append(belgium_response["loudness"])
    belgium_mode.append(belgium_response["mode"])
    belgium_speechiness.append(belgium_response["speechiness"])
    belgium_acousticness.append(belgium_response["acousticness"])
    belgium_instrumentalness.append(belgium_response["instrumentalness"])
    belgium_liveness.append(belgium_response["liveness"])
    belgium_valence.append(belgium_response["valence"])
    belgium_tempo.append(belgium_response["tempo"])
    belgium_id_num.append(belgium_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    belgium_duration_mins.append(round((belgium_response["duration_ms"] / 60000),2))
    belgium_time_signature.append(belgium_response["time_signature"])  

    # time.sleep(1)

belgium_dict = {}
#create a dictionary to hold data gathered from the api
belgium_dict = {
    "track_id": belgium_id_num,
    "danceability": belgium_danceability,
    "energy": belgium_energy,
    "key": belgium_key,
    "loudness": belgium_loudness,
    "mode": belgium_mode,
    "speechiness": belgium_speechiness,
    "acousticness": belgium_acousticness,
    "instrumentalness": belgium_instrumentalness,
    "liveness": belgium_liveness,
    "valence": belgium_valence,
    "tempo": belgium_tempo,
    "duration_mins": belgium_duration_mins,
    "time_signature": belgium_time_signature
}

#convert dictionary to a dataframe
belgium_info_df = pd.DataFrame(belgium_dict)
belgium_info_df.head()


belgium_tracks = d['belgium']
belgium_df = belgium_tracks.merge(belgium_info_df, how="inner")
belgium_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Belgium,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
1,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Belgium,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
2,4h9wh7iOZ0GGn8QVp4RAOB,OneRepublic,I Ain't Worried,Belgium,0.704,0.797,0,-5.927,1,0.0475,0.08260,0.000745,0.0546,0.825,139.994,2.47,4
3,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Belgium,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,2.79,4
4,3D29kjUyWxsT3jUUTtARVQ,Gazo,DIE,Belgium,0.695,0.630,8,-7.160,0,0.0350,0.22900,0.000000,0.1180,0.550,130.968,4.01,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1NFgyfNYN7ZlVeS0TUbCna,"Jinho 9, KATNUF, Jeriel Thunder, Bizzey",Interessant (feat. Bizzey),Belgium,0.588,0.658,11,-5.701,0,0.3170,0.28800,0.000000,0.1300,0.624,101.161,2.38,4
196,3Wrjm47oTz2sjIgck11l5e,Måneskin,Beggin',Belgium,0.714,0.800,11,-4.808,0,0.0504,0.12700,0.000000,0.3590,0.589,134.002,3.53,4
197,0V5cvmTKsYmF5FmGGEAfmS,Patrick Watson,Je te laisserai des mots,Belgium,0.303,0.187,2,-16.757,1,0.0356,0.98900,0.489000,0.1010,0.213,132.731,2.68,4
198,46vfSYsDJZaNQcD4YvT8GD,"Naps, GIMS",Best life,Belgium,0.919,0.779,7,-4.739,0,0.1470,0.25800,0.000000,0.0991,0.941,138.038,3.16,4


# Bolivia Top Chart Track Features

In [40]:
#empty lists to store song data
bolivia_danceability = []
bolivia_energy = []
bolivia_key = []
bolivia_loudness = []
bolivia_mode = []
bolivia_speechiness = []
bolivia_acousticness = []
bolivia_instrumentalness = []
bolivia_liveness = []
bolivia_valence = []
bolivia_tempo = []
bolivia_id_num = []
bolivia_duration_mins = []
bolivia_time_signature = []

for i in d['bolivia']["track_id"]:
    bolivia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    bolivia_danceability.append(bolivia_response["danceability"])
    bolivia_energy.append(bolivia_response["energy"])
    bolivia_key.append(bolivia_response["key"])
    bolivia_loudness.append(bolivia_response["loudness"])
    bolivia_mode.append(bolivia_response["mode"])
    bolivia_speechiness.append(bolivia_response["speechiness"])
    bolivia_acousticness.append(bolivia_response["acousticness"])
    bolivia_instrumentalness.append(bolivia_response["instrumentalness"])
    bolivia_liveness.append(bolivia_response["liveness"])
    bolivia_valence.append(bolivia_response["valence"])
    bolivia_tempo.append(bolivia_response["tempo"])
    bolivia_id_num.append(bolivia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    bolivia_duration_mins.append(round((bolivia_response["duration_ms"] / 60000),2))
    bolivia_time_signature.append(bolivia_response["time_signature"])  

    # time.sleep(1)

bolivia_dict = {}
#create a dictionary to hold data gathered from the api
bolivia_dict = {
    "track_id": bolivia_id_num,
    "danceability": bolivia_danceability,
    "energy": bolivia_energy,
    "key": bolivia_key,
    "loudness": bolivia_loudness,
    "mode": bolivia_mode,
    "speechiness": bolivia_speechiness,
    "acousticness": bolivia_acousticness,
    "instrumentalness": bolivia_instrumentalness,
    "liveness": bolivia_liveness,
    "valence": bolivia_valence,
    "tempo": bolivia_tempo,
    "duration_mins": bolivia_duration_mins,
    "time_signature": bolivia_time_signature
}

#convert dictionary to a dataframe
bolivia_info_df = pd.DataFrame(bolivia_dict)
bolivia_info_df.head()


bolivia_tracks = d['bolivia']
bolivia_df = bolivia_tracks.merge(bolivia_info_df, how="inner")
bolivia_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Bolivia,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Bolivia,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,3.32,4
2,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Bolivia,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,2.98,4
3,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Bolivia,0.801,0.475,7,-8.797,0,0.0516,0.14100,0.000017,0.0639,0.234,98.047,3.55,4
4,3k3NWokhRRkEPhCzPmV8TW,"Bad Bunny, Bomba Estéreo",Ojitos Lindos,Bolivia,0.647,0.686,3,-5.745,0,0.0413,0.08000,0.000001,0.5280,0.268,79.928,4.30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6qyS9qBy0mEk3qYaH8mPss,Kordhell,Murder In My Mind,Bolivia,0.712,0.972,10,-0.514,1,0.1120,0.00547,0.000071,0.1280,0.568,119.966,2.42,4
196,2qEoAz0i6yEz5dPggABcLH,"Fuego, Manuel Turizo, Duki",Una Vaina Loca,Bolivia,0.811,0.588,1,-3.161,1,0.0464,0.12800,0.000400,0.0929,0.553,107.022,3.77,4
197,4SSbtLDkepeOHAckM1FCBP,"El Alfa, Noriel, Trueno",LOS APARATOS,Bolivia,0.868,0.878,1,-3.294,1,0.0380,0.05090,0.000520,0.0857,0.899,112.987,3.88,4
198,1qfoq1qvZT7VwDVeekCJ97,Christian Nodal,De Los Besos Que Te Di,Bolivia,0.504,0.503,7,-3.313,1,0.0422,0.38800,0.000000,0.0608,0.709,195.593,2.79,4


# Brazil Top Chart Track Features

In [41]:
#empty lists to store song data
brazil_danceability = []
brazil_energy = []
brazil_key = []
brazil_loudness = []
brazil_mode = []
brazil_speechiness = []
brazil_acousticness = []
brazil_instrumentalness = []
brazil_liveness = []
brazil_valence = []
brazil_tempo = []
brazil_id_num = []
brazil_duration_mins = []
brazil_time_signature = []

for i in d['brazil']["track_id"]:
    brazil_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    brazil_danceability.append(brazil_response["danceability"])
    brazil_energy.append(brazil_response["energy"])
    brazil_key.append(brazil_response["key"])
    brazil_loudness.append(brazil_response["loudness"])
    brazil_mode.append(brazil_response["mode"])
    brazil_speechiness.append(brazil_response["speechiness"])
    brazil_acousticness.append(brazil_response["acousticness"])
    brazil_instrumentalness.append(brazil_response["instrumentalness"])
    brazil_liveness.append(brazil_response["liveness"])
    brazil_valence.append(brazil_response["valence"])
    brazil_tempo.append(brazil_response["tempo"])
    brazil_id_num.append(brazil_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    brazil_duration_mins.append(round((brazil_response["duration_ms"] / 60000),2))
    brazil_time_signature.append(brazil_response["time_signature"])  

    # time.sleep(1)

brazil_dict = {}
#create a dictionary to hold data gathered from the api
brazil_dict = {
    "track_id": brazil_id_num,
    "danceability": brazil_danceability,
    "energy": brazil_energy,
    "key": brazil_key,
    "loudness": brazil_loudness,
    "mode": brazil_mode,
    "speechiness": brazil_speechiness,
    "acousticness": brazil_acousticness,
    "instrumentalness": brazil_instrumentalness,
    "liveness": brazil_liveness,
    "valence": brazil_valence,
    "tempo": brazil_tempo,
    "duration_mins": brazil_duration_mins,
    "time_signature": brazil_time_signature
}

#convert dictionary to a dataframe
brazil_info_df = pd.DataFrame(brazil_dict)
brazil_info_df.head()


brazil_tracks = d['brazil']
brazil_df = brazil_tracks.merge(brazil_info_df, how="inner")
brazil_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,1OZ4uv7OAP24Ix6j3vojLt,"Mc Paiva ZS, MC Ryan SP, Kotim",Casei Com a Putaria,Brazil,0.585,0.600,10,-7.008,0,0.4360,0.1170,0.000000,0.0504,0.617,160.635,2.62,4
1,7qoIhutxU269Zqo9PG5IOj,"Pineapple StormTv, Chris MC, Salve Malak, Tz d...",Poesia Acústica 13,Brazil,0.572,0.579,10,-7.242,0,0.1280,0.4690,0.000000,0.1360,0.500,127.982,10.34,4
2,1ELsaYdj2NlxLvkx9g24xZ,"Guilherme & Benuto, Hugo & Guilherme",Haja Colírio (feat. Hugo & Guilherme) - Ao Vivo,Brazil,0.714,0.748,11,-4.605,1,0.0679,0.5900,0.000000,0.6260,0.698,105.031,2.89,4
3,3yHXEidSu4gYVRqKoI653O,"Guilherme & Benuto, Xand Avião",Assunto Delicado - Ao Vivo,Brazil,0.569,0.941,4,-4.135,0,0.3120,0.4010,0.000000,0.8910,0.608,73.776,3.01,4
4,4ASA1PZyWGbuc4d9N8OAcF,"Gustavo Mioto, Mari Fernandez",Eu Gosto Assim - Ao Vivo,Brazil,0.636,0.909,0,-3.685,1,0.0439,0.1490,0.000000,0.7190,0.610,154.189,2.48,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3JJXjB4TtfJMQSqVl1Ul9y,"MC BN, MC Guto VGS, Dj kl do abc",Eu Acho Que Vi um Gatinho,Brazil,0.947,0.695,10,-9.621,0,0.3590,0.7970,0.000249,0.2880,0.657,130.029,2.01,4
196,62ke5zFUJN6RvtXZgVH0F8,Paloma Faith,Only Love Can Hurt Like This,Brazil,0.566,0.885,8,-4.528,1,0.0818,0.0958,0.000100,0.3340,0.304,90.990,3.88,4
197,7kzFnjDEbftVgf7HL6xf9j,"Léo & Raphael, Jorge",Larga AÍ Pra Ver - Ao Vivo,Brazil,0.602,0.900,3,-3.843,1,0.1800,0.4610,0.000000,0.3890,0.781,149.775,2.66,4
198,7gVzhRCrLRUBEGdS7zNTp0,Zé Felipe,Malvada,Brazil,0.894,0.859,5,-2.981,0,0.0471,0.1590,0.000153,0.0750,0.888,134.947,2.43,4


# Bulgaria Top Chart Track Features

In [42]:
#empty lists to store song data
bulgaria_danceability = []
bulgaria_energy = []
bulgaria_key = []
bulgaria_loudness = []
bulgaria_mode = []
bulgaria_speechiness = []
bulgaria_acousticness = []
bulgaria_instrumentalness = []
bulgaria_liveness = []
bulgaria_valence = []
bulgaria_tempo = []
bulgaria_id_num = []
bulgaria_duration_mins = []
bulgaria_time_signature = []

for i in d['bulgaria']["track_id"]:
    bulgaria_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    bulgaria_danceability.append(bulgaria_response["danceability"])
    bulgaria_energy.append(bulgaria_response["energy"])
    bulgaria_key.append(bulgaria_response["key"])
    bulgaria_loudness.append(bulgaria_response["loudness"])
    bulgaria_mode.append(bulgaria_response["mode"])
    bulgaria_speechiness.append(bulgaria_response["speechiness"])
    bulgaria_acousticness.append(bulgaria_response["acousticness"])
    bulgaria_instrumentalness.append(bulgaria_response["instrumentalness"])
    bulgaria_liveness.append(bulgaria_response["liveness"])
    bulgaria_valence.append(bulgaria_response["valence"])
    bulgaria_tempo.append(bulgaria_response["tempo"])
    bulgaria_id_num.append(bulgaria_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    bulgaria_duration_mins.append(round((bulgaria_response["duration_ms"] / 60000),2))
    bulgaria_time_signature.append(bulgaria_response["time_signature"])  

    # time.sleep(1)

bulgaria_dict = {}
#create a dictionary to hold data gathered from the api
bulgaria_dict = {
    "track_id": bulgaria_id_num,
    "danceability": bulgaria_danceability,
    "energy": bulgaria_energy,
    "key": bulgaria_key,
    "loudness": bulgaria_loudness,
    "mode": bulgaria_mode,
    "speechiness": bulgaria_speechiness,
    "acousticness": bulgaria_acousticness,
    "instrumentalness": bulgaria_instrumentalness,
    "liveness": bulgaria_liveness,
    "valence": bulgaria_valence,
    "tempo": bulgaria_tempo,
    "duration_mins": bulgaria_duration_mins,
    "time_signature": bulgaria_time_signature
}

#convert dictionary to a dataframe
bulgaria_info_df = pd.DataFrame(bulgaria_dict)
bulgaria_info_df.head()


bulgaria_tracks = d['bulgaria']
bulgaria_df = bulgaria_tracks.merge(bulgaria_info_df, how="inner")
bulgaria_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Bulgaria,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
1,1Oxshg4204lRdcogzvdP4x,"Emilia, Galin",Alcohol,Bulgaria,0.829,0.910,1,-3.461,0,0.0567,0.16500,0.000000,0.0526,0.962,144.031,3.19,4
2,56dILgPraSNLKcd89hxXaq,Molec,7 Dni,Bulgaria,0.597,0.710,0,-5.706,0,0.4020,0.28300,0.000000,0.1170,0.478,89.245,3.82,4
3,2yjlYDiNiQkdxVqTlaSrlX,Nicki Minaj,Super Freaky Girl,Bulgaria,0.951,0.878,2,-2.565,1,0.2110,0.05090,0.000016,0.3210,0.923,133.014,2.85,4
4,5qX0V96jKJ6zC8UImlXKmO,"V:RGO, EMIL TRF, FYRE, 2bona",PARALIA,Bulgaria,0.817,0.647,9,-6.475,1,0.2430,0.14500,0.000000,0.1020,0.723,142.981,4.56,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6ARd0Qd8iunz6lNyYQZz4t,"Tita, Djena",Да си траят,Bulgaria,0.468,0.910,6,-2.073,1,0.3510,0.08650,0.000000,0.1780,0.494,167.951,3.81,4
196,2xql0pid3EUwW38AsywxhV,The Neighbourhood,Reflections,Bulgaria,0.648,0.594,11,-7.027,0,0.0293,0.11400,0.046200,0.1160,0.544,113.983,4.07,4
197,1JpqwDolccy7XGrFINYLYy,"Donika, Dessita",Никога твой,Bulgaria,0.693,0.875,0,-5.811,0,0.0510,0.00487,0.001900,0.0753,0.926,159.987,2.83,4
198,1NXDDexJAupRgVjkEdaGP8,Krisko,Balenciaga Kuchek,Bulgaria,0.587,0.950,1,-2.393,0,0.0543,0.05690,0.000000,0.0407,0.964,170.020,3.97,4


# Canada Top Chart Track Features

In [43]:
#empty lists to store song data
canada_danceability = []
canada_energy = []
canada_key = []
canada_loudness = []
canada_mode = []
canada_speechiness = []
canada_acousticness = []
canada_instrumentalness = []
canada_liveness = []
canada_valence = []
canada_tempo = []
canada_id_num = []
canada_duration_mins = []
canada_time_signature = []

for i in d['canada']["track_id"]:
    canada_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    canada_danceability.append(canada_response["danceability"])
    canada_energy.append(canada_response["energy"])
    canada_key.append(canada_response["key"])
    canada_loudness.append(canada_response["loudness"])
    canada_mode.append(canada_response["mode"])
    canada_speechiness.append(canada_response["speechiness"])
    canada_acousticness.append(canada_response["acousticness"])
    canada_instrumentalness.append(canada_response["instrumentalness"])
    canada_liveness.append(canada_response["liveness"])
    canada_valence.append(canada_response["valence"])
    canada_tempo.append(canada_response["tempo"])
    canada_id_num.append(canada_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    canada_duration_mins.append(round((canada_response["duration_ms"] / 60000),2))
    canada_time_signature.append(canada_response["time_signature"])  

    # time.sleep(1)

canada_dict = {}
#create a dictionary to hold data gathered from the api
canada_dict = {
    "track_id": canada_id_num,
    "danceability": canada_danceability,
    "energy": canada_energy,
    "key": canada_key,
    "loudness": canada_loudness,
    "mode": canada_mode,
    "speechiness": canada_speechiness,
    "acousticness": canada_acousticness,
    "instrumentalness": canada_instrumentalness,
    "liveness": canada_liveness,
    "valence": canada_valence,
    "tempo": canada_tempo,
    "duration_mins": canada_duration_mins,
    "time_signature": canada_time_signature
}

#convert dictionary to a dataframe
canada_info_df = pd.DataFrame(canada_dict)
canada_info_df.head()


canada_tracks = d['canada']
canada_df = canada_tracks.merge(canada_info_df, how="inner")
canada_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Canada,0.714,0.472,2,-7.375,1,0.0864,0.013000,0.000005,0.2660,0.238,131.121,2.62,4
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Canada,0.561,0.965,7,-3.673,0,0.0343,0.003830,0.000007,0.3710,0.304,128.040,2.92,4
2,4h9wh7iOZ0GGn8QVp4RAOB,OneRepublic,I Ain't Worried,Canada,0.704,0.797,0,-5.927,1,0.0475,0.082600,0.000745,0.0546,0.825,139.994,2.47,4
3,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,Canada,0.686,0.494,1,-7.093,1,0.0355,0.613000,0.000058,0.4020,0.700,168.946,3.87,4
4,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Canada,0.520,0.731,6,-5.338,0,0.0557,0.342000,0.001010,0.3110,0.662,173.930,2.79,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2MuWTIM3b0YEAskbeeFE1i,Metallica,Master of Puppets (Remastered),Canada,0.543,0.836,4,-9.110,0,0.0353,0.000647,0.431000,0.1530,0.560,105.173,8.59,4
196,7ef4DlsgrMEH11cDZd32M6,"Calvin Harris, Dua Lipa",One Kiss (with Dua Lipa),Canada,0.791,0.862,9,-3.240,0,0.1100,0.037000,0.000022,0.0814,0.592,123.994,3.58,4
197,3W9jnJBPYeihwNSXsr65f9,"Lil Mabu, DUSTY LOCANE",NO SNITCHING (with Dusty Locane),Canada,0.731,0.653,1,-8.767,1,0.3480,0.062700,0.000000,0.3480,0.424,155.943,2.67,4
198,18vXApRmJSgQ6wG2ll9AOg,Polo G,RAPSTAR,Canada,0.789,0.536,6,-6.862,1,0.2420,0.410000,0.000000,0.1290,0.437,81.039,2.77,4


In [44]:
#empty lists to store song data
chile_danceability = []
chile_energy = []
chile_key = []
chile_loudness = []
chile_mode = []
chile_speechiness = []
chile_acousticness = []
chile_instrumentalness = []
chile_liveness = []
chile_valence = []
chile_tempo = []
chile_id_num = []
chile_duration_mins = []
chile_time_signature = []

for i in d['chile']["track_id"]:
    chile_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    chile_danceability.append(chile_response["danceability"])
    chile_energy.append(chile_response["energy"])
    chile_key.append(chile_response["key"])
    chile_loudness.append(chile_response["loudness"])
    chile_mode.append(chile_response["mode"])
    chile_speechiness.append(chile_response["speechiness"])
    chile_acousticness.append(chile_response["acousticness"])
    chile_instrumentalness.append(chile_response["instrumentalness"])
    chile_liveness.append(chile_response["liveness"])
    chile_valence.append(chile_response["valence"])
    chile_tempo.append(chile_response["tempo"])
    chile_id_num.append(chile_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    chile_duration_mins.append(round((chile_response["duration_ms"] / 60000),2))
    chile_time_signature.append(chile_response["time_signature"])  

    # time.sleep(1)

chile_dict = {}
#create a dictionary to hold data gathered from the api
chile_dict = {
    "track_id": chile_id_num,
    "danceability": chile_danceability,
    "energy": chile_energy,
    "key": chile_key,
    "loudness": chile_loudness,
    "mode": chile_mode,
    "speechiness": chile_speechiness,
    "acousticness": chile_acousticness,
    "instrumentalness": chile_instrumentalness,
    "liveness": chile_liveness,
    "valence": chile_valence,
    "tempo": chile_tempo,
    "duration_mins": chile_duration_mins,
    "time_signature": chile_time_signature,
}

#convert dictionary to a dataframe
chile_info_df = pd.DataFrame(chile_dict)
chile_info_df.head()


chile_tracks = d['chile']
chile_df = chile_tracks.merge(chile_info_df, how="inner")
chile_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,0kTMK4gNFfLXaTb62w1UaJ,"Pablo Pesadilla, Polimá Westcoast, Nickoog Clk...",BABY OTAKU,Chile,0.835,0.747,9,-4.650,1,0.0830,0.04920,0.000000,0.0550,0.432,102.003,2.94,4
1,2pR6bipXfJySBhje19SW02,"Stars Music Chile, Cris Mj, Standly",Marisola,Chile,0.806,0.850,1,-4.840,1,0.0967,0.15500,0.000000,0.2010,0.728,95.014,3.07,4
2,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Chile,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,2.71,4
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Chile,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,3.32,4
4,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Chile,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,2.98,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4h9wh7iOZ0GGn8QVp4RAOB,OneRepublic,I Ain't Worried,Chile,0.704,0.797,0,-5.927,1,0.0475,0.08260,0.000745,0.0546,0.825,139.994,2.47,4
196,3NW1FV4c9UNsPLkFOqTfF5,"Nickoog Clk, Izahn, Standly",Mi Waxita,Chile,0.750,0.595,9,-9.292,1,0.0589,0.18900,0.000000,0.0741,0.403,90.042,3.63,4
197,76OGwb5RA9h4FxQPT33ekc,Rosa Linn,SNAP,Chile,0.565,0.636,0,-8.198,1,0.0638,0.10700,0.000010,0.4470,0.525,170.010,2.99,4
198,58ge6dfP91o9oXMzq3XkIS,Arctic Monkeys,505,Chile,0.520,0.852,0,-5.866,1,0.0543,0.00237,0.000058,0.0733,0.234,140.267,4.23,4


In [45]:
#empty lists to store song data
colombia_danceability = []
colombia_energy = []
colombia_key = []
colombia_loudness = []
colombia_mode = []
colombia_speechiness = []
colombia_acousticness = []
colombia_instrumentalness = []
colombia_liveness = []
colombia_valence = []
colombia_tempo = []
colombia_id_num = []
colombia_duration_mins = []
colombia_time_signature = []

for i in d['colombia']["track_id"]:
    colombia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    colombia_danceability.append(colombia_response["danceability"])
    colombia_energy.append(colombia_response["energy"])
    colombia_key.append(colombia_response["key"])
    colombia_loudness.append(colombia_response["loudness"])
    colombia_mode.append(colombia_response["mode"])
    colombia_speechiness.append(colombia_response["speechiness"])
    colombia_acousticness.append(colombia_response["acousticness"])
    colombia_instrumentalness.append(colombia_response["instrumentalness"])
    colombia_liveness.append(colombia_response["liveness"])
    colombia_valence.append(colombia_response["valence"])
    colombia_tempo.append(colombia_response["tempo"])
    colombia_id_num.append(colombia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    colombia_duration_mins.append(round((colombia_response["duration_ms"] / 60000),2))
    colombia_time_signature.append(colombia_response["time_signature"])  

    # time.sleep(1)

colombia_dict = {}
#create a dictionary to hold data gathered from the api
colombia_dict = {
    "track_id": colombia_id_num,
    "danceability": colombia_danceability,
    "energy": colombia_energy,
    "key": colombia_key,
    "loudness": colombia_loudness,
    "mode": colombia_mode,
    "speechiness": colombia_speechiness,
    "acousticness": colombia_acousticness,
    "instrumentalness": colombia_instrumentalness,
    "liveness": colombia_liveness,
    "valence": colombia_valence,
    "tempo": colombia_tempo,
    "duration_mins": colombia_duration_mins,
    "time_signature": colombia_time_signature,
}

#convert dictionary to a dataframe
colombia_info_df = pd.DataFrame(colombia_dict)
colombia_info_df.head()


colombia_tracks = d['colombia']
colombia_df = colombia_tracks.merge(colombia_info_df, how="inner")
colombia_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,2CeKVsFFXG4QzA415QygGb,Feid,Feliz Cumpleaños Ferxxo,Colombia,0.865,0.573,5,-5.501,1,0.0678,0.0863,0.000000,0.3050,0.563,94.999,2.60,4
1,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Colombia,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
2,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Colombia,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
3,3PpWCS95gDUDRPko3u1MZ9,Feid,Prohibidox,Colombia,0.647,0.801,1,-2.946,0,0.2490,0.0451,0.000000,0.0628,0.517,180.108,2.77,4
4,0T2pB7P1VdXPhLdQZ488uH,Feid,Normal,Colombia,0.709,0.560,4,-4.867,0,0.1710,0.0340,0.000000,0.3150,0.541,170.082,2.86,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,75UMtPC77vqKRwfSuYuNPO,Kevin Kaarl,San Lucas,Colombia,0.577,0.363,7,-12.378,1,0.0263,0.8600,0.000282,0.0868,0.265,91.511,4.12,4
196,2ctjDCCg1wHoQSjIJ8p6U4,Plan B,Candy,Colombia,0.897,0.864,11,-3.735,0,0.1470,0.2520,0.000000,0.5170,0.542,95.981,3.46,4
197,2HbmLkHkkI15eES8kpWRuI,Lalo Rodriguez,Ven Devórame Otra Vez,Colombia,0.643,0.602,5,-9.280,0,0.0438,0.6670,0.000086,0.2760,0.961,83.778,5.19,4
198,5dtzqRtiXKftAk6B6vxUmr,"Alvaro Diaz, Feid, Tainy",Llori Pari,Colombia,0.602,0.573,5,-7.388,1,0.0518,0.2820,0.000000,0.0854,0.319,174.048,3.58,4


In [46]:
#empty lists to store song data
costa_rica_danceability = []
costa_rica_energy = []
costa_rica_key = []
costa_rica_loudness = []
costa_rica_mode = []
costa_rica_speechiness = []
costa_rica_acousticness = []
costa_rica_instrumentalness = []
costa_rica_liveness = []
costa_rica_valence = []
costa_rica_tempo = []
costa_rica_id_num = []
costa_rica_duration_mins = []
costa_rica_time_signature = []

for i in d['costa_rica']["track_id"]:
    costa_rica_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    costa_rica_danceability.append(costa_rica_response["danceability"])
    costa_rica_energy.append(costa_rica_response["energy"])
    costa_rica_key.append(costa_rica_response["key"])
    costa_rica_loudness.append(costa_rica_response["loudness"])
    costa_rica_mode.append(costa_rica_response["mode"])
    costa_rica_speechiness.append(costa_rica_response["speechiness"])
    costa_rica_acousticness.append(costa_rica_response["acousticness"])
    costa_rica_instrumentalness.append(costa_rica_response["instrumentalness"])
    costa_rica_liveness.append(costa_rica_response["liveness"])
    costa_rica_valence.append(costa_rica_response["valence"])
    costa_rica_tempo.append(costa_rica_response["tempo"])
    costa_rica_id_num.append(costa_rica_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    costa_rica_duration_mins.append(round((costa_rica_response["duration_ms"] / 60000),2))
    costa_rica_time_signature.append(costa_rica_response["time_signature"])  

    # time.sleep(1)

costa_rica_dict = {}
#create a dictionary to hold data gathered from the api
costa_rica_dict = {
    "track_id": costa_rica_id_num,
    "danceability": costa_rica_danceability,
    "energy": costa_rica_energy,
    "key": costa_rica_key,
    "loudness": costa_rica_loudness,
    "mode": costa_rica_mode,
    "speechiness": costa_rica_speechiness,
    "acousticness": costa_rica_acousticness,
    "instrumentalness": costa_rica_instrumentalness,
    "liveness": costa_rica_liveness,
    "valence": costa_rica_valence,
    "tempo": costa_rica_tempo,
    "duration_mins": costa_rica_duration_mins,
    "time_signature": costa_rica_time_signature,
}

#convert dictionary to a dataframe
costa_rica_info_df = pd.DataFrame(costa_rica_dict)
costa_rica_info_df.head()


costa_rica_tracks = d['costa_rica']
costa_rica_df = costa_rica_tracks.merge(costa_rica_info_df, how="inner")
costa_rica_df['region'] = "Costa Rica"
costa_rica_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Costa Rica,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Costa Rica,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Costa Rica,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,2.98,4
3,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Costa Rica,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,4.06,4
4,3k3NWokhRRkEPhCzPmV8TW,"Bad Bunny, Bomba Estéreo",Ojitos Lindos,Costa Rica,0.647,0.686,3,-5.745,0,0.0413,0.0800,0.000001,0.5280,0.268,79.928,4.30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2gpQi3hbcUAcEG8m2dlgfB,Paulo Londra,Plan A,Costa Rica,0.583,0.834,0,-4.875,1,0.0444,0.0495,0.000000,0.0658,0.557,173.935,2.97,4
196,5jzEwSyyymBlf1fa1o39T2,"Mora, Jhay Cortez",512,Costa Rica,0.818,0.673,4,-7.542,0,0.1480,0.3480,0.000000,0.1720,0.695,87.961,3.23,4
197,2FSGUA0gFgGeQdprjtGM2M,"KAROL G, Mariah Angeliq",EL MAKINON,Costa Rica,0.635,0.706,5,-3.224,0,0.1250,0.2560,0.000000,0.1080,0.443,88.863,3.49,4
198,6j5LtifAnuTjTYvml61yFZ,Vicente Fernández,Un Millón de Primaveras,Costa Rica,0.730,0.341,3,-8.521,1,0.0578,0.6540,0.000181,0.0926,0.822,100.558,2.88,3


In [47]:
#empty lists to store song data
cyprus_danceability = []
cyprus_energy = []
cyprus_key = []
cyprus_loudness = []
cyprus_mode = []
cyprus_speechiness = []
cyprus_acousticness = []
cyprus_instrumentalness = []
cyprus_liveness = []
cyprus_valence = []
cyprus_tempo = []
cyprus_id_num = []
cyprus_duration_mins = []
cyprus_time_signature = []

for i in d['cyprus']["track_id"]:
    cyprus_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    cyprus_danceability.append(cyprus_response["danceability"])
    cyprus_energy.append(cyprus_response["energy"])
    cyprus_key.append(cyprus_response["key"])
    cyprus_loudness.append(cyprus_response["loudness"])
    cyprus_mode.append(cyprus_response["mode"])
    cyprus_speechiness.append(cyprus_response["speechiness"])
    cyprus_acousticness.append(cyprus_response["acousticness"])
    cyprus_instrumentalness.append(cyprus_response["instrumentalness"])
    cyprus_liveness.append(cyprus_response["liveness"])
    cyprus_valence.append(cyprus_response["valence"])
    cyprus_tempo.append(cyprus_response["tempo"])
    cyprus_id_num.append(cyprus_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    cyprus_duration_mins.append(round((cyprus_response["duration_ms"] / 60000),2))
    cyprus_time_signature.append(cyprus_response["time_signature"])  

    # time.sleep(1)

cyprus_dict = {}
#create a dictionary to hold data gathered from the api
cyprus_dict = {
    "track_id": cyprus_id_num,
    "danceability": cyprus_danceability,
    "energy": cyprus_energy,
    "key": cyprus_key,
    "loudness": cyprus_loudness,
    "mode": cyprus_mode,
    "speechiness": cyprus_speechiness,
    "acousticness": cyprus_acousticness,
    "instrumentalness": cyprus_instrumentalness,
    "liveness": cyprus_liveness,
    "valence": cyprus_valence,
    "tempo": cyprus_tempo,
    "duration_mins": cyprus_duration_mins,
    "time_signature": cyprus_time_signature,
}

#convert dictionary to a dataframe
cyprus_info_df = pd.DataFrame(cyprus_dict)
cyprus_info_df.head()


cyprus_tracks = d['cyprus']
cyprus_df = cyprus_tracks.merge(cyprus_info_df, how="inner")
cyprus_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Cyprus,0.714,0.472,2,-7.375,1,0.0864,0.013000,0.000005,0.2660,0.238,131.121,2.62,4
1,2wuJWqcn6bJWIJCmb83Yeu,SIDARTA,ERIMO,Cyprus,0.661,0.734,8,-4.356,0,0.1570,0.423000,0.001650,0.1500,0.680,127.897,3.57,5
2,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Cyprus,0.561,0.965,7,-3.673,0,0.0343,0.003830,0.000007,0.3710,0.304,128.040,2.92,4
3,3xo0AJj0gWlp51SqoGaGDv,RICTA,Emmonh,Cyprus,0.799,0.582,9,-8.300,0,0.2250,0.793000,0.000133,0.0930,0.230,100.028,3.94,4
4,5flH7FY4ledAQboO5a0XQ5,"UZI, Mavi",SENIN UGRUNA,Cyprus,0.785,0.638,1,-7.081,1,0.1090,0.357000,0.011800,0.0982,0.348,127.058,2.03,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1l3SxOyTfhdtHw9V4HGTzz,"SNIK, Strat",Sinialo,Cyprus,0.415,0.748,4,-9.262,0,0.1560,0.479000,0.000003,0.1930,0.487,100.693,2.52,3
196,2TktkzfozZifbQhXjT6I33,Omar Apollo,Evergreen (You Didn’t Deserve Me At All),Cyprus,0.697,0.336,9,-10.395,1,0.0424,0.598000,0.005280,0.1070,0.310,82.029,3.61,3
197,259d0kioJeWhOXnNBMyV0B,"Aspa, Mad Clip",Oh No,Cyprus,0.798,0.801,7,-4.827,0,0.1880,0.054500,0.000000,0.2890,0.506,104.970,2.67,4
198,3azJifCSqg9fRij2yKIbWz,Tory Lanez,The Color Violet,Cyprus,0.645,0.534,6,-10.800,0,0.0479,0.157000,0.000000,0.0863,0.463,105.020,3.77,4


In [48]:
#empty lists to store song data
czech_republic_danceability = []
czech_republic_energy = []
czech_republic_key = []
czech_republic_loudness = []
czech_republic_mode = []
czech_republic_speechiness = []
czech_republic_acousticness = []
czech_republic_instrumentalness = []
czech_republic_liveness = []
czech_republic_valence = []
czech_republic_tempo = []
czech_republic_id_num = []
czech_republic_duration_mins = []
czech_republic_time_signature = []

for i in d['czech_republic']["track_id"]:
    czech_republic_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    czech_republic_danceability.append(czech_republic_response["danceability"])
    czech_republic_energy.append(czech_republic_response["energy"])
    czech_republic_key.append(czech_republic_response["key"])
    czech_republic_loudness.append(czech_republic_response["loudness"])
    czech_republic_mode.append(czech_republic_response["mode"])
    czech_republic_speechiness.append(czech_republic_response["speechiness"])
    czech_republic_acousticness.append(czech_republic_response["acousticness"])
    czech_republic_instrumentalness.append(czech_republic_response["instrumentalness"])
    czech_republic_liveness.append(czech_republic_response["liveness"])
    czech_republic_valence.append(czech_republic_response["valence"])
    czech_republic_tempo.append(czech_republic_response["tempo"])
    czech_republic_id_num.append(czech_republic_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    czech_republic_duration_mins.append(round((czech_republic_response["duration_ms"] / 60000),2))
    czech_republic_time_signature.append(czech_republic_response["time_signature"])  

    # time.sleep(1)

czech_republic_dict = {}
#create a dictionary to hold data gathered from the api
czech_republic_dict = {
    "track_id": czech_republic_id_num,
    "danceability": czech_republic_danceability,
    "energy": czech_republic_energy,
    "key": czech_republic_key,
    "loudness": czech_republic_loudness,
    "mode": czech_republic_mode,
    "speechiness": czech_republic_speechiness,
    "acousticness": czech_republic_acousticness,
    "instrumentalness": czech_republic_instrumentalness,
    "liveness": czech_republic_liveness,
    "valence": czech_republic_valence,
    "tempo": czech_republic_tempo,
    "duration_mins": czech_republic_duration_mins,
    "time_signature": czech_republic_time_signature
}

#convert dictionary to a dataframe
czech_republic_info_df = pd.DataFrame(czech_republic_dict)
czech_republic_info_df.head()


czech_republic_tracks = d['czech_republic']
czech_republic_df = czech_republic_tracks.merge(czech_republic_info_df, how="inner")
czech_republic_df['region'] = "Czech Republic"
czech_republic_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Czech Republic,0.714,0.472,2,-7.375,1,0.0864,0.013000,0.000005,0.2660,0.2380,131.121,2.62,4
1,4ToxkJ4VvWs026R3QfHDI3,Calin,Santé,Czech Republic,0.604,0.649,2,-8.458,0,0.0524,0.400000,0.273000,0.1860,0.2730,120.111,1.90,4
2,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Czech Republic,0.561,0.965,7,-3.673,0,0.0343,0.003830,0.000007,0.3710,0.3040,128.040,2.92,4
3,2sXnat46y10FtAbW2A0ira,Calin,Hannah Montana,Czech Republic,0.583,0.784,6,-6.708,0,0.0882,0.062100,0.000045,0.1710,0.7440,135.080,2.64,4
4,4h9wh7iOZ0GGn8QVp4RAOB,OneRepublic,I Ain't Worried,Czech Republic,0.704,0.797,0,-5.927,1,0.0475,0.082600,0.000745,0.0546,0.8250,139.994,2.47,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0JzG1wJuB4v9UPkSE7LMXi,Montell Fish,Hotel,Czech Republic,0.228,0.427,0,-8.739,0,0.0363,0.752000,0.016700,0.5920,0.1920,180.345,3.29,4
196,3Oww84xrmgjyr5J1ilOmAf,"Luude, Colin Hay",Down Under (feat. Colin Hay),Czech Republic,0.308,0.860,11,-4.111,0,0.1680,0.011600,0.000828,0.2770,0.0387,171.835,2.65,4
197,30bqVoKjX479ab90a8Pafp,Lil Peep,Star Shopping,Czech Republic,0.585,0.471,4,-9.934,0,0.0616,0.096700,0.018400,0.1150,0.3230,93.099,2.37,4
198,7EkWXAI1wn8Ii883ecd9xr,Surf Curse,Freaks,Czech Republic,0.345,0.941,9,-9.918,1,0.0466,0.000038,0.633000,0.0511,0.4070,179.626,2.45,4


In [49]:
#empty lists to store song data
denmark_danceability = []
denmark_energy = []
denmark_key = []
denmark_loudness = []
denmark_mode = []
denmark_speechiness = []
denmark_acousticness = []
denmark_instrumentalness = []
denmark_liveness = []
denmark_valence = []
denmark_tempo = []
denmark_id_num = []
denmark_duration_mins = []
denmark_time_signature = []

for i in d['denmark']["track_id"]:
    denmark_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    denmark_danceability.append(denmark_response["danceability"])
    denmark_energy.append(denmark_response["energy"])
    denmark_key.append(denmark_response["key"])
    denmark_loudness.append(denmark_response["loudness"])
    denmark_mode.append(denmark_response["mode"])
    denmark_speechiness.append(denmark_response["speechiness"])
    denmark_acousticness.append(denmark_response["acousticness"])
    denmark_instrumentalness.append(denmark_response["instrumentalness"])
    denmark_liveness.append(denmark_response["liveness"])
    denmark_valence.append(denmark_response["valence"])
    denmark_tempo.append(denmark_response["tempo"])
    denmark_id_num.append(denmark_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    denmark_duration_mins.append(round((denmark_response["duration_ms"] / 60000),2))
    denmark_time_signature.append(denmark_response["time_signature"])  

    # time.sleep(1)

denmark_dict = {}
#create a dictionary to hold data gathered from the api
denmark_dict = {
    "track_id": denmark_id_num,
    "danceability": denmark_danceability,
    "energy": denmark_energy,
    "key": denmark_key,
    "loudness": denmark_loudness,
    "mode": denmark_mode,
    "speechiness": denmark_speechiness,
    "acousticness": denmark_acousticness,
    "instrumentalness": denmark_instrumentalness,
    "liveness": denmark_liveness,
    "valence": denmark_valence,
    "tempo": denmark_tempo,
    "duration_mins": denmark_duration_mins,
    "time_signature": denmark_time_signature
}

#convert dictionary to a dataframe
denmark_info_df = pd.DataFrame(denmark_dict)
denmark_info_df.head()


denmark_tracks = d['denmark']
denmark_df = denmark_tracks.merge(denmark_info_df, how="inner")
denmark_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Denmark,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
1,5iqWgicPaUYWjxORp5iagj,Tobias Rahim,Flyvende Faduma,Denmark,0.682,0.751,7,-6.030,1,0.0337,0.09160,0.000030,0.3290,0.558,74.993,3.30,4
2,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Denmark,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
3,63Dymm6U4WHLRy87e8GBbo,Lamin,"Gode Dage, Gode Drinks",Denmark,0.789,0.703,0,-7.401,1,0.3670,0.04480,0.000000,0.1400,0.482,112.025,2.33,4
4,7bgknc0sSmkIWc2am7QDEv,Tobias Rahim,Feberdrømmer Xx Dubai,Denmark,0.710,0.628,11,-6.661,0,0.0483,0.19700,0.000008,0.1080,0.462,135.990,3.69,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5QO79kh1waicV47BqGRL3g,The Weeknd,Save Your Tears,Denmark,0.680,0.826,0,-5.487,1,0.0309,0.02120,0.000012,0.5430,0.644,118.051,3.59,4
196,2VxeLyX666F8uXCJ0dZF8B,"Lady Gaga, Bradley Cooper",Shallow,Denmark,0.572,0.385,7,-6.362,1,0.0308,0.37100,0.000000,0.2310,0.323,95.799,3.60,4
197,2He7ZIvNyBo188a6hvS0zT,Tessa,Ben,Denmark,0.897,0.536,1,-5.612,0,0.3080,0.39800,0.000000,0.3380,0.710,92.984,2.94,4
198,2BOqDYLOJBiMOXShCV1neZ,Calum Scott,Dancing On My Own,Denmark,0.681,0.174,1,-8.745,1,0.0315,0.83700,0.000034,0.0983,0.231,112.672,4.34,4


In [50]:
#empty lists to store song data
dominican_republic_danceability = []
dominican_republic_energy = []
dominican_republic_key = []
dominican_republic_loudness = []
dominican_republic_mode = []
dominican_republic_speechiness = []
dominican_republic_acousticness = []
dominican_republic_instrumentalness = []
dominican_republic_liveness = []
dominican_republic_valence = []
dominican_republic_tempo = []
dominican_republic_id_num = []
dominican_republic_duration_mins = []
dominican_republic_time_signature = []

for i in d['dominican_republic']["track_id"]:
    dominican_republic_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    dominican_republic_danceability.append(dominican_republic_response["danceability"])
    dominican_republic_energy.append(dominican_republic_response["energy"])
    dominican_republic_key.append(dominican_republic_response["key"])
    dominican_republic_loudness.append(dominican_republic_response["loudness"])
    dominican_republic_mode.append(dominican_republic_response["mode"])
    dominican_republic_speechiness.append(dominican_republic_response["speechiness"])
    dominican_republic_acousticness.append(dominican_republic_response["acousticness"])
    dominican_republic_instrumentalness.append(dominican_republic_response["instrumentalness"])
    dominican_republic_liveness.append(dominican_republic_response["liveness"])
    dominican_republic_valence.append(dominican_republic_response["valence"])
    dominican_republic_tempo.append(dominican_republic_response["tempo"])
    dominican_republic_id_num.append(dominican_republic_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    dominican_republic_duration_mins.append(round((dominican_republic_response["duration_ms"] / 60000),2))
    dominican_republic_time_signature.append(dominican_republic_response["time_signature"])  

    # time.sleep(1)

dominican_republic_dict = {}
#create a dictionary to hold data gathered from the api
dominican_republic_dict = {
    "track_id": dominican_republic_id_num,
    "danceability": dominican_republic_danceability,
    "energy": dominican_republic_energy,
    "key": dominican_republic_key,
    "loudness": dominican_republic_loudness,
    "mode": dominican_republic_mode,
    "speechiness": dominican_republic_speechiness,
    "acousticness": dominican_republic_acousticness,
    "instrumentalness": dominican_republic_instrumentalness,
    "liveness": dominican_republic_liveness,
    "valence": dominican_republic_valence,
    "tempo": dominican_republic_tempo,
    "duration_mins": dominican_republic_duration_mins,
    "time_signature": dominican_republic_time_signature
}

#convert dictionary to a dataframe
dominican_republic_info_df = pd.DataFrame(dominican_republic_dict)
dominican_republic_info_df.head()


dominican_republic_tracks = d['dominican_republic']
dominican_republic_df = dominican_republic_tracks.merge(dominican_republic_info_df, how="inner")
dominican_republic_df['region'] = "Dominican Republic"
dominican_republic_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Dominican Republic,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,2.98,4
1,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Dominican Republic,0.801,0.475,7,-8.797,0,0.0516,0.1410,0.000017,0.0639,0.234,98.047,3.55,4
2,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Dominican Republic,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
3,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Dominican Republic,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,4.06,4
4,6Xom58OOXk2SoU711L2IXO,Bad Bunny,Moscow Mule,Dominican Republic,0.804,0.674,5,-5.453,0,0.0333,0.2940,0.000001,0.1150,0.292,99.968,4.10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,76OGwb5RA9h4FxQPT33ekc,Rosa Linn,SNAP,Dominican Republic,0.565,0.636,0,-8.198,1,0.0638,0.1070,0.000010,0.4470,0.525,170.010,2.99,4
196,78QcVmzxS0wIwuC2rpAauI,"Rauw Alejandro, Mr. Naisgai",2/Catorce,Dominican Republic,0.768,0.560,3,-3.497,1,0.0859,0.7090,0.000791,0.1150,0.372,112.084,3.43,3
197,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,Dominican Republic,0.686,0.494,1,-7.093,1,0.0355,0.6130,0.000058,0.4020,0.700,168.946,3.87,4
198,2zknaYPaPICrQ5b04Y5oDP,Dj Concon,Dj Concon Vol1,Dominican Republic,0.634,0.126,9,-20.696,1,0.0342,0.7380,0.841000,0.1120,0.335,110.092,1.55,3


In [51]:
#empty lists to store song data
ecuador_danceability = []
ecuador_energy = []
ecuador_key = []
ecuador_loudness = []
ecuador_mode = []
ecuador_speechiness = []
ecuador_acousticness = []
ecuador_instrumentalness = []
ecuador_liveness = []
ecuador_valence = []
ecuador_tempo = []
ecuador_id_num = []
ecuador_duration_mins = []
ecuador_time_signature = []

for i in d['ecuador']["track_id"]:
    ecuador_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    ecuador_danceability.append(ecuador_response["danceability"])
    ecuador_energy.append(ecuador_response["energy"])
    ecuador_key.append(ecuador_response["key"])
    ecuador_loudness.append(ecuador_response["loudness"])
    ecuador_mode.append(ecuador_response["mode"])
    ecuador_speechiness.append(ecuador_response["speechiness"])
    ecuador_acousticness.append(ecuador_response["acousticness"])
    ecuador_instrumentalness.append(ecuador_response["instrumentalness"])
    ecuador_liveness.append(ecuador_response["liveness"])
    ecuador_valence.append(ecuador_response["valence"])
    ecuador_tempo.append(ecuador_response["tempo"])
    ecuador_id_num.append(ecuador_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    ecuador_duration_mins.append(round((ecuador_response["duration_ms"] / 60000),2))
    ecuador_time_signature.append(ecuador_response["time_signature"])  

    # time.sleep(1)

ecuador_dict = {}
#create a dictionary to hold data gathered from the api
ecuador_dict = {
    "track_id": ecuador_id_num,
    "danceability": ecuador_danceability,
    "energy": ecuador_energy,
    "key": ecuador_key,
    "loudness": ecuador_loudness,
    "mode": ecuador_mode,
    "speechiness": ecuador_speechiness,
    "acousticness": ecuador_acousticness,
    "instrumentalness": ecuador_instrumentalness,
    "liveness": ecuador_liveness,
    "valence": ecuador_valence,
    "tempo": ecuador_tempo,
    "duration_mins": ecuador_duration_mins,
    "time_signature": ecuador_time_signature
}

#convert dictionary to a dataframe
ecuador_info_df = pd.DataFrame(ecuador_dict)
ecuador_info_df.head()


ecuador_tracks = d['ecuador']
ecuador_df = ecuador_tracks.merge(ecuador_info_df, how="inner")
ecuador_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Ecuador,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Ecuador,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,2CeKVsFFXG4QzA415QygGb,Feid,Feliz Cumpleaños Ferxxo,Ecuador,0.865,0.573,5,-5.501,1,0.0678,0.0863,0.000000,0.3050,0.563,94.999,2.60,4
3,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Ecuador,0.801,0.475,7,-8.797,0,0.0516,0.1410,0.000017,0.0639,0.234,98.047,3.55,4
4,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Ecuador,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,2.98,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3b1IQflSLrgzYQPGFzI9cl,Juanes,Es Por Ti,Ecuador,0.696,0.758,4,-6.076,1,0.0264,0.1700,0.000033,0.1890,0.859,129.943,4.18,4
196,1BYZxKSf0aTxp8ZFoeyM3d,girl in red,we fell in love in october,Ecuador,0.566,0.366,7,-12.808,1,0.0280,0.1130,0.181000,0.1550,0.237,129.959,3.07,4
197,6VAS7m73rDopQe7BdrFMpD,"Bizarrap, Tiago PZK","Tiago PZK: Bzrp Music Sessions, Vol. 48",Ecuador,0.657,0.821,8,-3.060,0,0.3200,0.4590,0.000000,0.1030,0.752,96.561,2.67,4
198,2C6eINAA8cO6e5a8PjnWsK,Feid,AMOR DE MI VIDA,Ecuador,0.735,0.674,6,-5.629,0,0.1240,0.1800,0.000000,0.1080,0.561,171.883,3.16,4


In [52]:
#empty lists to store song data
egypt_danceability = []
egypt_energy = []
egypt_key = []
egypt_loudness = []
egypt_mode = []
egypt_speechiness = []
egypt_acousticness = []
egypt_instrumentalness = []
egypt_liveness = []
egypt_valence = []
egypt_tempo = []
egypt_id_num = []
egypt_duration_mins = []
egypt_time_signature = []

for i in d['egypt']["track_id"]:
    egypt_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    egypt_danceability.append(egypt_response["danceability"])
    egypt_energy.append(egypt_response["energy"])
    egypt_key.append(egypt_response["key"])
    egypt_loudness.append(egypt_response["loudness"])
    egypt_mode.append(egypt_response["mode"])
    egypt_speechiness.append(egypt_response["speechiness"])
    egypt_acousticness.append(egypt_response["acousticness"])
    egypt_instrumentalness.append(egypt_response["instrumentalness"])
    egypt_liveness.append(egypt_response["liveness"])
    egypt_valence.append(egypt_response["valence"])
    egypt_tempo.append(egypt_response["tempo"])
    egypt_id_num.append(egypt_response["id"])    
    #duration recorded in api in milliseconds, converting to minutes here
    egypt_duration_mins.append(round((egypt_response["duration_ms"] / 60000),2))
    egypt_time_signature.append(egypt_response["time_signature"])  

    # time.sleep(1)

egypt_dict = {}
#create a dictionary to hold data gathered from the api
egypt_dict = {
    "track_id": egypt_id_num,
    "danceability": egypt_danceability,
    "energy": egypt_energy,
    "key": egypt_key,
    "loudness": egypt_loudness,
    "mode": egypt_mode,
    "speechiness": egypt_speechiness,
    "acousticness": egypt_acousticness,
    "instrumentalness": egypt_instrumentalness,
    "liveness": egypt_liveness,
    "valence": egypt_valence,
    "tempo": egypt_tempo,
    "duration_mins": egypt_duration_mins,
    "time_signature": egypt_time_signature
}

#convert dictionary to a dataframe
egypt_info_df = pd.DataFrame(egypt_dict)
egypt_info_df.head()


egypt_tracks = d['egypt']
egypt_df = egypt_tracks.merge(egypt_info_df, how="inner")
egypt_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,13E54FVYV7fzEfZhtrKn0K,"Cairokee, Marwan Pablo",Tarantino,Egypt,0.710,0.559,3,-7.478,0,0.0781,0.17500,0.000000,0.0460,0.573,141.922,3.03,4
1,2BHxg3zP8IdRvvWPVavzqB,"Wegz, Ash",Amira,Egypt,0.755,0.594,6,-7.316,0,0.0422,0.12000,0.000002,0.0666,0.629,117.948,4.49,4
2,1GKqff6G0zabRInKN9Vz1S,Cairokee,James Dean,Egypt,0.795,0.669,11,-7.434,0,0.0315,0.12400,0.000458,0.1880,0.540,113.982,4.08,4
3,5kdlxZ5skOYY3VK8RlfEIZ,Wegz,البخت,Egypt,0.780,0.590,3,-7.770,0,0.1320,0.48700,0.000011,0.1150,0.660,104.977,3.44,4
4,4vIbws1xp082bv0zl8vLaJ,Cairokee,Basrah w Atooh,Egypt,0.740,0.507,3,-8.764,0,0.1180,0.37300,0.000201,0.2390,0.551,103.993,3.96,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1NkW18VV3QVOCuoVQsBC84,Tamer Ashour,Baoul Aady,Egypt,0.533,0.600,3,-9.079,0,0.0381,0.19100,0.000332,0.1230,0.384,139.991,5.53,4
196,6qyS9qBy0mEk3qYaH8mPss,Kordhell,Murder In My Mind,Egypt,0.712,0.972,10,-0.514,1,0.1120,0.00547,0.000071,0.1280,0.568,119.966,2.42,4
197,75FEaRjZTKLhTrFGsfMUXR,Kate Bush,Running Up That Hill (A Deal With God) - 2018 ...,Egypt,0.629,0.547,10,-13.123,0,0.0550,0.72000,0.003140,0.0604,0.197,108.375,4.98,4
198,29KS7kBSU77PrYGB6RVR5O,"Marshmello, Amr Diab",Bayen Habeit,Egypt,0.791,0.761,5,-5.760,0,0.0392,0.23100,0.000626,0.3670,0.797,111.663,2.32,4


In [53]:
#empty lists to store song data
el_salvador_danceability = []
el_salvador_energy = []
el_salvador_key = []
el_salvador_loudness = []
el_salvador_mode = []
el_salvador_speechiness = []
el_salvador_acousticness = []
el_salvador_instrumentalness = []
el_salvador_liveness = []
el_salvador_valence = []
el_salvador_tempo = []
el_salvador_id_num = []
el_salvador_duration_mins = []
el_salvador_time_signature = []

for i in d['el_salvador']["track_id"]:
    el_salvador_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    el_salvador_danceability.append(el_salvador_response["danceability"])
    el_salvador_energy.append(el_salvador_response["energy"])
    el_salvador_key.append(el_salvador_response["key"])
    el_salvador_loudness.append(el_salvador_response["loudness"])
    el_salvador_mode.append(el_salvador_response["mode"])
    el_salvador_speechiness.append(el_salvador_response["speechiness"])
    el_salvador_acousticness.append(el_salvador_response["acousticness"])
    el_salvador_instrumentalness.append(el_salvador_response["instrumentalness"])
    el_salvador_liveness.append(el_salvador_response["liveness"])
    el_salvador_valence.append(el_salvador_response["valence"])
    el_salvador_tempo.append(el_salvador_response["tempo"])
    el_salvador_id_num.append(el_salvador_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    el_salvador_duration_mins.append(round((el_salvador_response["duration_ms"] / 60000),2))
    el_salvador_time_signature.append(el_salvador_response["time_signature"])  

    # time.sleep(1)

el_salvador_dict = {}
#create a dictionary to hold data gathered from the api
el_salvador_dict = {
    "track_id": el_salvador_id_num,
    "danceability": el_salvador_danceability,
    "energy": el_salvador_energy,
    "key": el_salvador_key,
    "loudness": el_salvador_loudness,
    "mode": el_salvador_mode,
    "speechiness": el_salvador_speechiness,
    "acousticness": el_salvador_acousticness,
    "instrumentalness": el_salvador_instrumentalness,
    "liveness": el_salvador_liveness,
    "valence": el_salvador_valence,
    "tempo": el_salvador_tempo,
    "duration_mins": el_salvador_duration_mins,
    "time_signature": el_salvador_time_signature
}

#convert dictionary to a dataframe
el_salvador_info_df = pd.DataFrame(el_salvador_dict)
el_salvador_info_df.head()


el_salvador_tracks = d['el_salvador']
el_salvador_df = el_salvador_tracks.merge(el_salvador_info_df, how="inner")
el_salvador_df['region'] = "El Salvador"
el_salvador_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,El Salvador,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,2.98,4
1,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,El Salvador,0.801,0.475,7,-8.797,0,0.0516,0.1410,0.000017,0.0639,0.234,98.047,3.55,4
2,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,El Salvador,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",El Salvador,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
4,3k3NWokhRRkEPhCzPmV8TW,"Bad Bunny, Bomba Estéreo",Ojitos Lindos,El Salvador,0.647,0.686,3,-5.745,0,0.0413,0.0800,0.000001,0.5280,0.268,79.928,4.30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3JCaq3KDSROg3TXhdiDa9n,"Jhayco, Anuel AA",Ley Seca,El Salvador,0.759,0.842,8,-3.758,1,0.0960,0.1010,0.000000,0.1340,0.584,105.059,4.39,4
196,4eecTlhTsAR8JMQXZmkbiT,"Dímelo Flow, Sech, Dalex, Justin Quiles, Arcan...",Se Le Ve,El Salvador,0.764,0.756,7,-4.153,0,0.0909,0.1180,0.000014,0.2260,0.651,167.772,4.55,4
197,4NUt1fcnO9aQAELBgXq3Kr,Alejandro Fernández,Me Dediqué a Perderte,El Salvador,0.642,0.442,8,-9.115,1,0.0740,0.4910,0.000002,0.1020,0.465,148.010,3.86,4
198,3WLXf7K7mIGEBtFq41afce,Jerry Rivera,Amores Como el Nuestro,El Salvador,0.698,0.872,0,-2.160,1,0.0456,0.5830,0.009460,0.0848,0.620,179.556,5.04,4


In [54]:
#empty lists to store song data
estonia_danceability = []
estonia_energy = []
estonia_key = []
estonia_loudness = []
estonia_mode = []
estonia_speechiness = []
estonia_acousticness = []
estonia_instrumentalness = []
estonia_liveness = []
estonia_valence = []
estonia_tempo = []
estonia_id_num = []
estonia_duration_mins = []
estonia_time_signature = []

for i in d['estonia']["track_id"]:
    estonia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    estonia_danceability.append(estonia_response["danceability"])
    estonia_energy.append(estonia_response["energy"])
    estonia_key.append(estonia_response["key"])
    estonia_loudness.append(estonia_response["loudness"])
    estonia_mode.append(estonia_response["mode"])
    estonia_speechiness.append(estonia_response["speechiness"])
    estonia_acousticness.append(estonia_response["acousticness"])
    estonia_instrumentalness.append(estonia_response["instrumentalness"])
    estonia_liveness.append(estonia_response["liveness"])
    estonia_valence.append(estonia_response["valence"])
    estonia_tempo.append(estonia_response["tempo"])
    estonia_id_num.append(estonia_response["id"])   
    #duration recorded in api in milliseconds, converting to minutes here
    estonia_duration_mins.append(round((estonia_response["duration_ms"] / 60000),2))
    estonia_time_signature.append(estonia_response["time_signature"])  

    # time.sleep(1)

estonia_dict = {}
#create a dictionary to hold data gathered from the api
estonia_dict = {
    "track_id": estonia_id_num,
    "danceability": estonia_danceability,
    "energy": estonia_energy,
    "key": estonia_key,
    "loudness": estonia_loudness,
    "mode": estonia_mode,
    "speechiness": estonia_speechiness,
    "acousticness": estonia_acousticness,
    "instrumentalness": estonia_instrumentalness,
    "liveness": estonia_liveness,
    "valence": estonia_valence,
    "tempo": estonia_tempo,
    "duration_mins": estonia_duration_mins,
    "time_signature": estonia_time_signature
}

#convert dictionary to a dataframe
estonia_info_df = pd.DataFrame(estonia_dict)
estonia_info_df.head()


estonia_tracks = d['estonia']
estonia_df = estonia_tracks.merge(estonia_info_df, how="inner")
estonia_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Estonia,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Estonia,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
2,4tRhRLBxIZ34Iw0eCuiC03,southstar,Miss You,Estonia,0.663,0.559,6,-12.876,0,0.0391,0.13600,0.089400,0.1080,0.296,145.007,3.44,4
3,41XLcwECxYj7tokaF2a7sA,nublu,kastehein,Estonia,0.554,0.800,11,-6.798,0,0.0585,0.03470,0.098600,0.1210,0.270,121.932,2.70,4
4,39JofJHEtg8I4fSyo7Imft,"Eliza Rose, Interplanetary Criminal",B.O.T.A. (Baddest Of Them All) - Edit,Estonia,0.736,0.906,0,-7.589,1,0.0480,0.16400,0.585000,0.1060,0.698,137.001,3.78,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,11iIikXxC6NP0Ma8vMD27x,Vundabar,Alien Blues,Estonia,0.467,0.757,3,-5.093,1,0.0330,0.08070,0.908000,0.0876,0.433,82.282,2.60,4
196,3pzr7iKQpV9QjLGcLcd2ic,"uglystephan, Scally Milano",Заберу,Estonia,0.764,0.700,2,-4.462,0,0.1170,0.11300,0.000000,0.1330,0.273,145.011,2.04,4
197,3ZCTVFBt2Brf31RLEnCkWJ,Billie Eilish,everything i wanted,Estonia,0.704,0.225,6,-14.454,0,0.0994,0.90200,0.657000,0.1060,0.243,120.006,4.09,4
198,6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,Estonia,0.548,0.816,0,-4.209,1,0.0465,0.12200,0.000000,0.3350,0.557,95.390,2.90,4


In [55]:
#empty lists to store song data
finland_danceability = []
finland_energy = []
finland_key = []
finland_loudness = []
finland_mode = []
finland_speechiness = []
finland_acousticness = []
finland_instrumentalness = []
finland_liveness = []
finland_valence = []
finland_tempo = []
finland_id_num = []
finland_duration_mins = []
finland_time_signature = []

for i in d['finland']["track_id"]:
    finland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    finland_danceability.append(finland_response["danceability"])
    finland_energy.append(finland_response["energy"])
    finland_key.append(finland_response["key"])
    finland_loudness.append(finland_response["loudness"])
    finland_mode.append(finland_response["mode"])
    finland_speechiness.append(finland_response["speechiness"])
    finland_acousticness.append(finland_response["acousticness"])
    finland_instrumentalness.append(finland_response["instrumentalness"])
    finland_liveness.append(finland_response["liveness"])
    finland_valence.append(finland_response["valence"])
    finland_tempo.append(finland_response["tempo"])
    finland_id_num.append(finland_response["id"])   
    #duration recorded in api in milliseconds, converting to minutes here
    finland_duration_mins.append(round((finland_response["duration_ms"] / 60000),2))
    finland_time_signature.append(finland_response["time_signature"])  

    # time.sleep(1)

finland_dict = {}
#create a dictionary to hold data gathered from the api
finland_dict = {
    "track_id": finland_id_num,
    "danceability": finland_danceability,
    "energy": finland_energy,
    "key": finland_key,
    "loudness": finland_loudness,
    "mode": finland_mode,
    "speechiness": finland_speechiness,
    "acousticness": finland_acousticness,
    "instrumentalness": finland_instrumentalness,
    "liveness": finland_liveness,
    "valence": finland_valence,
    "tempo": finland_tempo,
    "duration_mins": finland_duration_mins,
    "time_signature": finland_time_signature,
}

#convert dictionary to a dataframe
finland_info_df = pd.DataFrame(finland_dict)
finland_info_df.head()


finland_tracks = d['finland']
finland_df = finland_tracks.merge(finland_info_df, how="inner")
finland_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,6qverLcFcryBhJBgomIZjw,"Turisti, Elastinen",Tippa-T,Finland,0.771,0.658,2,-5.205,1,0.0912,0.02610,0.000000,0.2730,0.490,99.993,2.28,4
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Finland,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
2,2G1gCuldjtW9xcN1n1yenB,"Isac Elliot, Sexmane",Makso Mitä Makso,Finland,0.704,0.818,1,-4.710,0,0.0318,0.03240,0.020500,0.3560,0.909,129.962,2.76,4
3,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Finland,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4
4,2h9FYDFYIScQ4dHtsL70gn,"Joe L, Costi",DEEP DOWN (feat. Costi),Finland,0.845,0.735,2,-6.618,1,0.0335,0.08410,0.008200,0.1080,0.584,120.020,3.08,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4rsTtGSlm50nQk4ZYKq0Dq,Madboiali,Allez Allez Allez,Finland,0.618,0.933,0,-3.624,0,0.0527,0.29900,0.000000,0.1280,0.442,135.974,2.61,4
196,2WFgAIWF83jJuWunwSk3mX,YOUNGHEARTED,Silloin,Finland,0.672,0.404,7,-9.107,1,0.0447,0.33800,0.000000,0.0954,0.331,126.917,3.37,4
197,3dp8H2alKT2fTrSBKhQoTW,VIIVI,Hyväuskoinen,Finland,0.594,0.456,6,-7.853,1,0.0329,0.12100,0.000000,0.0995,0.245,132.665,3.44,4
198,29B7useM6iMR6PFnNJfJTn,Haloo Helsinki!,Pelikaani,Finland,0.412,0.489,1,-7.282,0,0.0477,0.11100,0.000001,0.2180,0.201,94.992,4.64,4


In [56]:
#empty lists to store song data
france_danceability = []
france_energy = []
france_key = []
france_loudness = []
france_mode = []
france_speechiness = []
france_acousticness = []
france_instrumentalness = []
france_liveness = []
france_valence = []
france_tempo = []
france_id_num = []
france_duration_mins = []
france_time_signature = []

for i in d['france']["track_id"]:
    france_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    france_danceability.append(france_response["danceability"])
    france_energy.append(france_response["energy"])
    france_key.append(france_response["key"])
    france_loudness.append(france_response["loudness"])
    france_mode.append(france_response["mode"])
    france_speechiness.append(france_response["speechiness"])
    france_acousticness.append(france_response["acousticness"])
    france_instrumentalness.append(france_response["instrumentalness"])
    france_liveness.append(france_response["liveness"])
    france_valence.append(france_response["valence"])
    france_tempo.append(france_response["tempo"])
    france_id_num.append(france_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    france_duration_mins.append(round((france_response["duration_ms"] / 60000),2))
    france_time_signature.append(france_response["time_signature"])  

    # time.sleep(1)

france_dict = {}
#create a dictionary to hold data gathered from the api
france_dict = {
    "track_id": france_id_num,
    "danceability": france_danceability,
    "energy": france_energy,
    "key": france_key,
    "loudness": france_loudness,
    "mode": france_mode,
    "speechiness": france_speechiness,
    "acousticness": france_acousticness,
    "instrumentalness": france_instrumentalness,
    "liveness": france_liveness,
    "valence": france_valence,
    "tempo": france_tempo,
    "duration_mins": france_duration_mins,
    "time_signature": france_time_signature
}

#convert dictionary to a dataframe
france_info_df = pd.DataFrame(france_dict)
france_info_df.head()


france_tracks = d['france']
france_df = france_tracks.merge(france_info_df, how="inner")
france_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3D29kjUyWxsT3jUUTtARVQ,Gazo,DIE,France,0.695,0.630,8,-7.160,0,0.0350,0.22900,0.000000,0.1180,0.550,130.968,4.01,4
1,6PrB4KpvuoYWtwjPC5a44S,"Soolking, Niska",Balader,France,0.603,0.802,4,-5.089,0,0.1110,0.30600,0.000051,0.1240,0.787,130.121,2.94,4
2,4ZpIuzx91EAPK3VimONbfB,"ZEG P, Hamza, SCH",FADE UP,France,0.758,0.599,4,-5.990,0,0.0877,0.57000,0.000001,0.1290,0.557,129.960,3.65,4
3,6OZwia8loN0aPS0vTvsBjR,"Alonzo, Ninho, Naps",TOUT VA BIEN (feat. Ninho & Naps),France,0.660,0.720,3,-5.874,0,0.0753,0.30100,0.000000,0.2260,0.628,99.937,3.22,4
4,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),France,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2DuTgYkwiaIK6rZ0D34n9g,"Dosseh, Lacrim",Amsterdam,France,0.783,0.863,4,-3.359,0,0.0611,0.20100,0.000003,0.1500,0.567,138.048,2.81,4
196,43yYuYeSOOupBHOkH9pRtF,Lomepal,Évidemment,France,0.640,0.715,10,-7.041,0,0.0945,0.67500,0.000063,0.1150,0.397,92.878,3.31,4
197,41DFvj092Ab3wcUJXTCRqG,"Prince Waly, Freeze corleone",Balotelli (feat. Freeze Corleone),France,0.779,0.470,0,-10.336,1,0.4060,0.50800,0.000000,0.1050,0.521,112.050,3.49,4
198,368KS9UNJ8cft7RJ1USwb9,Ziak,Raspoutine,France,0.910,0.658,5,-8.843,1,0.3820,0.21500,0.000000,0.0910,0.616,145.023,1.98,4


In [57]:
#empty lists to store song data
germany_danceability = []
germany_energy = []
germany_key = []
germany_loudness = []
germany_mode = []
germany_speechiness = []
germany_acousticness = []
germany_instrumentalness = []
germany_liveness = []
germany_valence = []
germany_tempo = []
germany_id_num = []
germany_duration_mins = []
germany_time_signature = []

for i in d['germany']["track_id"]:
    germany_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    germany_danceability.append(germany_response["danceability"])
    germany_energy.append(germany_response["energy"])
    germany_key.append(germany_response["key"])
    germany_loudness.append(germany_response["loudness"])
    germany_mode.append(germany_response["mode"])
    germany_speechiness.append(germany_response["speechiness"])
    germany_acousticness.append(germany_response["acousticness"])
    germany_instrumentalness.append(germany_response["instrumentalness"])
    germany_liveness.append(germany_response["liveness"])
    germany_valence.append(germany_response["valence"])
    germany_tempo.append(germany_response["tempo"])
    germany_id_num.append(germany_response["id"])    
    #duration recorded in api in milliseconds, converting to minutes here
    germany_duration_mins.append(round((germany_response["duration_ms"] / 60000),2))
    germany_time_signature.append(germany_response["time_signature"])  

    # time.sleep(1)

germany_dict = {}
#create a dictionary to hold data gathered from the api
germany_dict = {
    "track_id": germany_id_num,
    "danceability": germany_danceability,
    "energy": germany_energy,
    "key": germany_key,
    "loudness": germany_loudness,
    "mode": germany_mode,
    "speechiness": germany_speechiness,
    "acousticness": germany_acousticness,
    "instrumentalness": germany_instrumentalness,
    "liveness": germany_liveness,
    "valence": germany_valence,
    "tempo": germany_tempo,
    "duration_mins": germany_duration_mins,
    "time_signature": germany_time_signature
}

#convert dictionary to a dataframe
germany_info_df = pd.DataFrame(germany_dict)
germany_info_df.head()


germany_tracks = d['germany']
germany_df = germany_tracks.merge(germany_info_df, how="inner")
germany_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,7sJNeWTXkZCEC7OnFMpGWL,"Luciano, Aitch, BIA",Bamba (feat. Aitch & BIA),Germany,0.797,0.791,10,-6.281,1,0.3300,0.1190,0.000000,0.1130,0.828,138.026,3.39,4
1,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),Germany,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,2.62,4
2,1Fb8gCHRkmNAaN8vytd5vg,"Luciano, Central Cee",West Connect (feat. Central Cee),Germany,0.920,0.571,10,-7.133,0,0.4940,0.0681,0.000000,0.0969,0.465,139.990,3.54,4
3,70Xfh3baxA4xusCiV5B5jg,"Jamule, FOURTY",KISSENSCHLACHT,Germany,0.910,0.447,1,-5.580,1,0.0688,0.1010,0.000377,0.1380,0.315,125.021,2.85,4
4,6yeZM8gyykjaf8w0GqRfK4,Nina Chuba,Wildberry Lillet,Germany,0.890,0.521,6,-7.048,0,0.1690,0.0780,0.000000,0.0958,0.646,139.981,2.20,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,78xy2KmJ9LzgLltj3eO0aq,HoodBlaq,Barrio,Germany,0.678,0.628,10,-6.971,0,0.0722,0.2610,0.000000,0.1180,0.182,127.005,3.36,4
196,4xkOaSrkexMciUUogZKVTS,"Eminem, Nate Dogg",Till I Collapse,Germany,0.548,0.847,1,-3.237,1,0.1860,0.0622,0.000000,0.0816,0.100,171.447,4.96,4
197,1WCEAGGRD066z2Q89ObXTq,"David Guetta, Becky Hill, Ella Henderson",Crazy What Love Can Do,Germany,0.601,0.713,4,-3.758,0,0.0449,0.0282,0.000000,0.1580,0.464,122.872,2.83,4
198,7iL6o9tox1zgHpKUfh9vuC,50 Cent,In Da Club,Germany,0.899,0.713,6,-2.752,0,0.3660,0.2550,0.000000,0.0708,0.777,90.051,3.22,4


In [58]:
#empty lists to store song data
greece_danceability = []
greece_energy = []
greece_key = []
greece_loudness = []
greece_mode = []
greece_speechiness = []
greece_acousticness = []
greece_instrumentalness = []
greece_liveness = []
greece_valence = []
greece_tempo = []
greece_id_num = []
greece_duration_mins = []
greece_time_signature = []

for i in d['greece']["track_id"]:
    greece_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    greece_danceability.append(greece_response["danceability"])
    greece_energy.append(greece_response["energy"])
    greece_key.append(greece_response["key"])
    greece_loudness.append(greece_response["loudness"])
    greece_mode.append(greece_response["mode"])
    greece_speechiness.append(greece_response["speechiness"])
    greece_acousticness.append(greece_response["acousticness"])
    greece_instrumentalness.append(greece_response["instrumentalness"])
    greece_liveness.append(greece_response["liveness"])
    greece_valence.append(greece_response["valence"])
    greece_tempo.append(greece_response["tempo"])
    greece_id_num.append(greece_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    greece_duration_mins.append(round((greece_response["duration_ms"] / 60000),2))
    greece_time_signature.append(greece_response["time_signature"])  

    # time.sleep(1)

greece_dict = {}
#create a dictionary to hold data gathered from the api
greece_dict = {
    "track_id": greece_id_num,
    "danceability": greece_danceability,
    "energy": greece_energy,
    "key": greece_key,
    "loudness": greece_loudness,
    "mode": greece_mode,
    "speechiness": greece_speechiness,
    "acousticness": greece_acousticness,
    "instrumentalness": greece_instrumentalness,
    "liveness": greece_liveness,
    "valence": greece_valence,
    "tempo": greece_tempo,
    "duration_mins": greece_duration_mins,
    "time_signature": greece_time_signature
}

#convert dictionary to a dataframe
greece_info_df = pd.DataFrame(greece_dict)
greece_info_df.head()


greece_tracks = d['greece']
greece_df = greece_tracks.merge(greece_info_df, how="inner")
greece_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,2wuJWqcn6bJWIJCmb83Yeu,SIDARTA,ERIMO,Greece,0.661,0.734,8,-4.356,0,0.1570,0.423000,0.001650,0.150,0.6800,127.897,3.57,5
1,3xo0AJj0gWlp51SqoGaGDv,RICTA,Emmonh,Greece,0.799,0.582,9,-8.300,0,0.2250,0.793000,0.000133,0.093,0.2300,100.028,3.94,4
2,6XK6R1ir3k0iPkwGbmVIEV,"FLY LO, Mike G",Crystal,Greece,0.706,0.716,0,-5.263,1,0.1300,0.221000,0.000000,0.157,0.0798,100.025,2.65,4
3,5dKi5wRWzTPWHmUxpq96Hf,"SIDARTA, Beyond",AKOMI,Greece,0.661,0.610,2,-7.939,1,0.0677,0.429000,0.000000,0.156,0.2170,95.020,2.95,4
4,1OGTkqFF8EmYZvDLuHAXsw,"Rack, Beyond",AMOR,Greece,0.772,0.749,1,-8.113,1,0.0856,0.233000,0.142000,0.111,0.3430,121.976,3.25,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,3xcSTAxSTwfx0A4Ly68dfW,"DJ.Silence, RICTA",Leyla,Greece,0.528,0.460,1,-6.772,1,0.0987,0.283000,0.000037,0.468,0.0903,106.864,2.97,4
196,3F5CgOj3wFlRv51JsHbxhe,"Drake, 21 Savage",Jimmy Cooks (feat. 21 Savage),Greece,0.529,0.673,0,-4.711,1,0.1750,0.000307,0.000002,0.093,0.3660,165.921,3.64,4
197,05WQrdexAfPXOTDJEGueYF,"Saske, Mike G, Nuki",MAVRA LEFTA,Greece,0.702,0.826,4,-8.690,0,0.1730,0.379000,0.000025,0.364,0.7520,132.955,3.22,4
198,54ipXppHLA8U4yqpOFTUhr,Imagine Dragons,Bones,Greece,0.772,0.750,5,-3.670,0,0.0455,0.020100,0.000000,0.074,0.5870,114.061,2.75,4


In [59]:
#empty lists to store song data
guatemala_danceability = []
guatemala_energy = []
guatemala_key = []
guatemala_loudness = []
guatemala_mode = []
guatemala_speechiness = []
guatemala_acousticness = []
guatemala_instrumentalness = []
guatemala_liveness = []
guatemala_valence = []
guatemala_tempo = []
guatemala_id_num = []
guatemala_duration_mins = []
guatemala_time_signature = []

for i in d['guatemala']["track_id"]:
    guatemala_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    guatemala_danceability.append(guatemala_response["danceability"])
    guatemala_energy.append(guatemala_response["energy"])
    guatemala_key.append(guatemala_response["key"])
    guatemala_loudness.append(guatemala_response["loudness"])
    guatemala_mode.append(guatemala_response["mode"])
    guatemala_speechiness.append(guatemala_response["speechiness"])
    guatemala_acousticness.append(guatemala_response["acousticness"])
    guatemala_instrumentalness.append(guatemala_response["instrumentalness"])
    guatemala_liveness.append(guatemala_response["liveness"])
    guatemala_valence.append(guatemala_response["valence"])
    guatemala_tempo.append(guatemala_response["tempo"])
    guatemala_id_num.append(guatemala_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    guatemala_duration_mins.append(round((guatemala_response["duration_ms"] / 60000),2))
    guatemala_time_signature.append(guatemala_response["time_signature"])  

    # time.sleep(1)

guatemala_dict = {}
#create a dictionary to hold data gathered from the api
guatemala_dict = {
    "track_id": guatemala_id_num,
    "danceability": guatemala_danceability,
    "energy": guatemala_energy,
    "key": guatemala_key,
    "loudness": guatemala_loudness,
    "mode": guatemala_mode,
    "speechiness": guatemala_speechiness,
    "acousticness": guatemala_acousticness,
    "instrumentalness": guatemala_instrumentalness,
    "liveness": guatemala_liveness,
    "valence": guatemala_valence,
    "tempo": guatemala_tempo,
    "duration_mins": guatemala_duration_mins,
    "time_signature": guatemala_time_signature
}

#convert dictionary to a dataframe
guatemala_info_df = pd.DataFrame(guatemala_dict)
guatemala_info_df.head()


guatemala_tracks = d['guatemala']
guatemala_df = guatemala_tracks.merge(guatemala_info_df, how="inner")
guatemala_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Guatemala,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,2.71,4
1,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Guatemala,0.801,0.475,7,-8.797,0,0.0516,0.14100,0.000017,0.0639,0.234,98.047,3.55,4
2,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Guatemala,0.911,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005,2.98,4
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Guatemala,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,3.32,4
4,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Guatemala,0.650,0.715,5,-5.198,0,0.2530,0.09930,0.000291,0.1260,0.187,106.672,4.06,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Guatemala,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4
196,1JSTJqkT5qHq8MDJnJbRE1,The Police,Every Breath You Take,Guatemala,0.820,0.452,1,-9.796,1,0.0348,0.54300,0.002940,0.0714,0.740,117.401,4.23,4
197,7jtQIBanIiJOMS6RyCx6jZ,Tom Odell,Another Love,Guatemala,0.442,0.538,4,-8.550,0,0.0451,0.71700,0.000029,0.0900,0.139,122.758,4.07,4
198,6lFM8kkmpffm0aq2gzEtEK,Aventura,Dile al Amor,Guatemala,0.824,0.612,11,-9.616,1,0.0372,0.07860,0.000000,0.3200,0.945,123.998,3.81,4


In [60]:
#empty lists to store song data
honduras_danceability = []
honduras_energy = []
honduras_key = []
honduras_loudness = []
honduras_mode = []
honduras_speechiness = []
honduras_acousticness = []
honduras_instrumentalness = []
honduras_liveness = []
honduras_valence = []
honduras_tempo = []
honduras_id_num = []
honduras_duration_mins = []
honduras_time_signature = []

for i in d['honduras']["track_id"]:
    honduras_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    honduras_danceability.append(honduras_response["danceability"])
    honduras_energy.append(honduras_response["energy"])
    honduras_key.append(honduras_response["key"])
    honduras_loudness.append(honduras_response["loudness"])
    honduras_mode.append(honduras_response["mode"])
    honduras_speechiness.append(honduras_response["speechiness"])
    honduras_acousticness.append(honduras_response["acousticness"])
    honduras_instrumentalness.append(honduras_response["instrumentalness"])
    honduras_liveness.append(honduras_response["liveness"])
    honduras_valence.append(honduras_response["valence"])
    honduras_tempo.append(honduras_response["tempo"])
    honduras_id_num.append(honduras_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    honduras_duration_mins.append(round((honduras_response["duration_ms"] / 60000),2))
    honduras_time_signature.append(honduras_response["time_signature"])  

    # time.sleep(1)

honduras_dict = {}
#create a dictionary to hold data gathered from the api
honduras_dict = {
    "track_id": honduras_id_num,
    "danceability": honduras_danceability,
    "energy": honduras_energy,
    "key": honduras_key,
    "loudness": honduras_loudness,
    "mode": honduras_mode,
    "speechiness": honduras_speechiness,
    "acousticness": honduras_acousticness,
    "instrumentalness": honduras_instrumentalness,
    "liveness": honduras_liveness,
    "valence": honduras_valence,
    "tempo": honduras_tempo,
    "duration_mins": honduras_duration_mins,
    "time_signature": honduras_time_signature
}

#convert dictionary to a dataframe
honduras_info_df = pd.DataFrame(honduras_dict)
honduras_info_df.head()


honduras_tracks = d['honduras']
honduras_df = honduras_tracks.merge(honduras_info_df, how="inner")
honduras_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5Eax0qFko2dh7Rl2lYs3bx,Bad Bunny,Efecto,Honduras,0.801,0.475,7,-8.797,0,0.0516,0.1410,0.000017,0.0639,0.234,98.047,3.55,4
1,6Sq7ltF9Qa7SNFBsV5Cogx,"Bad Bunny, Chencho Corleone",Me Porto Bonito,Honduras,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,2.98,4
2,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Honduras,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
3,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,Honduras,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,4.06,4
4,3k3NWokhRRkEPhCzPmV8TW,"Bad Bunny, Bomba Estéreo",Ojitos Lindos,Honduras,0.647,0.686,3,-5.745,0,0.0413,0.0800,0.000001,0.5280,0.268,79.928,4.30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,16ONd70tvWAaLIc6hcLzaG,Christian Nodal,Aquí Abajo,Honduras,0.638,0.437,4,-3.857,1,0.0332,0.2820,0.000000,0.0894,0.618,147.880,2.40,3
196,2q9udNV9NK0BL3q9p6TLxf,"ChiChi Peralta, Jandy Feliz",Procura,Honduras,0.805,0.757,8,-7.460,1,0.0355,0.2790,0.000839,0.1010,0.973,102.140,4.52,4
197,0ynhqjP6gIxoaGTWibGhjc,"Rochy RD, El Chuky De Lewa",Uva Bombom,Honduras,0.630,0.920,1,-1.564,0,0.4000,0.1230,0.000137,0.0403,0.963,227.895,2.27,4
198,1BYZxKSf0aTxp8ZFoeyM3d,girl in red,we fell in love in october,Honduras,0.566,0.366,7,-12.808,1,0.0280,0.1130,0.181000,0.1550,0.237,129.959,3.07,4


In [61]:
#empty lists to store song data
hong_kong_danceability = []
hong_kong_energy = []
hong_kong_key = []
hong_kong_loudness = []
hong_kong_mode = []
hong_kong_speechiness = []
hong_kong_acousticness = []
hong_kong_instrumentalness = []
hong_kong_liveness = []
hong_kong_valence = []
hong_kong_tempo = []
hong_kong_id_num = []
hong_kong_duration_mins = []
hong_kong_time_signature = []

for i in d['hong_kong']["track_id"]:
    hong_kong_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    hong_kong_danceability.append(hong_kong_response["danceability"])
    hong_kong_energy.append(hong_kong_response["energy"])
    hong_kong_key.append(hong_kong_response["key"])
    hong_kong_loudness.append(hong_kong_response["loudness"])
    hong_kong_mode.append(hong_kong_response["mode"])
    hong_kong_speechiness.append(hong_kong_response["speechiness"])
    hong_kong_acousticness.append(hong_kong_response["acousticness"])
    hong_kong_instrumentalness.append(hong_kong_response["instrumentalness"])
    hong_kong_liveness.append(hong_kong_response["liveness"])
    hong_kong_valence.append(hong_kong_response["valence"])
    hong_kong_tempo.append(hong_kong_response["tempo"])
    hong_kong_id_num.append(hong_kong_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    hong_kong_duration_mins.append(round((hong_kong_response["duration_ms"] / 60000),2))
    hong_kong_time_signature.append(hong_kong_response["time_signature"])  

    # time.sleep(1)

hong_kong_dict = {}
#create a dictionary to hold data gathered from the api
hong_kong_dict = {
    "track_id": hong_kong_id_num,
    "danceability": hong_kong_danceability,
    "energy": hong_kong_energy,
    "key": hong_kong_key,
    "loudness": hong_kong_loudness,
    "mode": hong_kong_mode,
    "speechiness": hong_kong_speechiness,
    "acousticness": hong_kong_acousticness,
    "instrumentalness": hong_kong_instrumentalness,
    "liveness": hong_kong_liveness,
    "valence": hong_kong_valence,
    "tempo": hong_kong_tempo,
    "duration_mins": hong_kong_duration_mins,
    "time_signature": hong_kong_time_signature
}

#convert dictionary to a dataframe
hong_kong_info_df = pd.DataFrame(hong_kong_dict)
hong_kong_info_df.head()


hong_kong_tracks = d['hong_kong']
hong_kong_df = hong_kong_tracks.merge(hong_kong_info_df, how="inner")
hong_kong_df

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,172oAbLGpgeCsukno7oBRe,Keung To,風雨不改 (電影《阿媽有咗第二個》主題曲),Hong_kong,0.319,0.198,0,-11.611,1,0.0402,0.9420,0.000009,0.0817,0.287,62.925,5.13,4
1,11IqNbLOD4s4nVYSuEttFR,Keung To,"Dear My Friend,",Hong_kong,0.646,0.452,0,-8.152,1,0.0307,0.7230,0.000000,0.1170,0.266,94.566,3.87,4
2,1dBroK67DSaaEE5kvkVjzG,Keung To,作品的說話,Hong_kong,0.635,0.389,5,-13.106,0,0.0638,0.4430,0.000000,0.1140,0.667,79.991,4.21,4
3,2lMVTk3I232dE7jnkufOSt,MC 張天賦,老派約會之必要,Hong_kong,0.679,0.563,10,-7.009,1,0.0334,0.5030,0.000000,0.1020,0.631,107.014,3.73,4
4,1UJYWfzyPjRwmlCawgJroV,Anson Lo 盧瀚霆,King Kong,Hong_kong,0.683,0.885,2,-2.827,1,0.0883,0.1400,0.000000,0.1370,0.285,114.017,3.78,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6kZfjIUmJqQAosGKyOcOIk,"Jacky Cheung, Anita Mui","相愛很難 - 電影""男人四十""歌曲",Hong_kong,0.495,0.559,0,-8.888,1,0.0470,0.6130,0.000000,0.1410,0.322,134.508,3.72,4
196,6HMtHNpW6YPi1hrw9tgF8P,Lizzo,About Damn Time,Hong_kong,0.836,0.743,10,-6.305,0,0.0653,0.0974,0.000000,0.3350,0.723,108.965,3.20,4
197,1h8dkllZVVtKhFJ2bsiuBC,Tyson Yoshi,Christy,Hong_kong,0.711,0.432,7,-7.770,1,0.0537,0.5150,0.000000,0.0743,0.144,80.047,2.90,4
198,6YI4OXOVtZ29Rc7GxOI4sY,Jace Chan,沒有無緣無故的恨,Hong_kong,0.647,0.405,4,-9.208,1,0.0594,0.7340,0.000000,0.3460,0.166,123.982,3.77,4
